# Airbnb Reviews Pipeline - MiniLM + SST-2

**Goal:** Extract negative, issue-tagged sentences from Airbnb reviews and produce actionable aggregations (property + host).

## Pipeline stages
1) Load + filter Italy
2) Split reviews blob → review chunks (review-level)
3) Split review chunks → sentences (English-only, min_words, smart dedup)
4) Top-K per property prefilter (reduce volume before embeddings)
5) Similarity (MiniLM) on candidate unique sentences and Apply similarity thresholds → issue candidates
7) Sentiment (SST-2) on issue-candidates (sharded + resume)
8) Join (issue × negative) → final issues (unique + occurrences)
9) Aggregations + evidence tables
10) LLM recommendations API on selected properties.

## Locked decisions
- Sentence filters: English-only (ASCII), `min_words=5`, smart normalization + dedup by `sentence_norm`
- Similarity: MiniLM computed once (no second pass)
- Similarity thresholds: `ISSUE_SIM_THRESHOLD=0.4`, `TOP2_GAP_MAX=0.04` (Top2 only if very close)
- Sentiment: SST-2, keep if `neg_prob >= 0.40`
- Outputs: final issues tables + LLM recommendations 


In [0]:
storage_account = "lab94290"
container = "airbnb"

In [0]:
sas_token="sp=rle&st=2025-12-24T17:37:04Z&se=2026-02-28T01:52:04Z&spr=https&sv=2024-11-04&sr=c&sig=a0lx%2BS6PuS%2FvJ9Tbt4NKdCJHLE9d1Y1D6vpE1WKFQtk%3D"
sas_token = sas_token.lstrip('?')
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{storage_account}.dfs.core.windows.net", sas_token)

In [0]:
path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/airbnb_1_12_parquet"

airbnb = spark.read.parquet(path)
display(airbnb.limit(5))

name price image description category availability discount reviews ratings seller_info breadcrumbs location lat long guests pets_allowed description_items category_rating house_rules details highlights arrangement_details amenities images available_dates url final_url listing_title property_id listing_name location_details description_by_sections description_html location_details_html is_supperhost host_number_of_reviews host_rating hosts_year host_response_rate is_guest_favorite travel_details pricing_details total_price currency cancellation_policy property_number_of_reviews country postcode_map_url host_image host_details Rental unit in Broadbeach · ★4.96 · 2 bedrooms · 2 beds · 1 bath 238 https://a0.muscache.com/pictures/50ddeaed-5302-4d0c-8200-163b2cddd071.jpg Our exceptionally stylish two bedroom, one bathroom LUXURY holiday apartment is located in the ORACLE T2, right in the heart of beautiful Broadbeach! Every corner of this gorgeous apartment overflows with style, luxury and thoughtful touches. Relax on the ocean facing balcony located directly above the manicured garden terrace and take in the relaxing views and sea breezes. This property is all class and guaranteed to satisfy even the fussiest Airbnb guest!The spaceThe photos speak for themselves. They are a true, accurate and up to date representation of our immaculately presented, squeaky clean, apartment. The furnishings are practical and comfortable and the linen and bedding are soft and luxurious. The master king bedroom opens out onto the large balcony. The second bedroom is a small internal room with a comfortable high quality queen sized bed. The second bedroom does not have a window. Each bedroom has an internal sliding door which provides access to the two way bathroom. Our apartment has been styled with a relaxing holiday in mind.Guest accessPoolSpaGymSauna and steam roomBBQ and entertaining areasGardensSecure off street parking for ONE vehicleAccess to facilities is not guaranteed and may be interrupted by routine and unexpected maintenance schedules.Other things to noteThis property accommodates up to FOUR people only. Children aged two and older are included in the four people rule. No exceptions. Infants UNDER the age of two are very welcome at our place and may stay at no additional charge however we do not provide linen or bedding. Let us know if you would like the link to a local business who hires out baby equipment (travel cots, strollers, high chairs).CHECK IN Available between 3pm and 9pm.Self check in is NOT available at this property. After hours check ins are NOT available. A 3pm check in is not guaranteed and depends on cleaning schedules. CHECK IN APPOINTMENTOur lovely host welcomes you in person on arrival. The booking guest is required to make contact with us via the booking message thread prior to their arrival day to arrange a check in time. A host will not be onsite to meet you unless you have scheduled a check in time. We always do our best to fit in with your travel schedule. IMPORTANTKeys and security fobs will only be issued to the person who made the booking. ID will be required. This is not negotiable. Please read the HOUSE RULES and CANCELLATION POLICY prior to booking our place. We recommend taking out travel insurance to cover any losses which may occur if you need to cancel your booking for an unforeseen reason. NO SMOKING - smoking is NOT permitted anywhere at this complex.CHECK IN / CHECK OUT - NOT AVAILABLE* Christmas Day* New Years Day * Easter SundayEARLY CHECK IN - We will always accommodate an early check in if our place is available. CHECK OUT by 11am - Late check outs are not available and hold up our cleaning team. WHAT WE INCLUDE IN OUR STARTER PACKOur starter pack is designed to help get you started on your holiday and may not last the entire trip. If you are with us for more than two nights, you may need to re stock items at the local supermarket. Woolworths supermarket is located right next door for any additi

In [0]:
from pyspark.sql import functions as F

df = airbnb.withColumn("country_from_location",
                       F.trim(F.element_at(F.split(F.col("location"), ","), -1)))

df_italy = df.filter(F.upper(F.col("country_from_location")) == "ITALY")

print("[Italy] rows:", df_italy.count())
print("[Italy] unique properties:", df_italy.select("property_id").distinct().count())

[Italy] rows: 98935
[Italy] unique properties: 92941


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import (
    StructType, StructField, StringType, FloatType, BooleanType
)
import pandas as pd
import numpy as np
from datetime import datetime

OUTPUT_BASE = "dbfs:/FileStore/airbnb"
RUN_ID = "ITALY_1"
print(f"[INFO] RUN_ID={RUN_ID}")

def save_table_and_path(df, table_name: str, mode: str = "overwrite"):
    # keep as metastore table (for spark.table usage)
    df.write.mode(mode).format("delta").saveAsTable(table_name)

    # save a Delta copy to DBFS path
    out_path = f"{OUTPUT_BASE}/{table_name}"
    df.write.mode(mode).format("delta").save(out_path)
    print(f"[OK] Saved table={table_name} | path={out_path}")

# Column names
PROPERTY_COL   = "property_id"
SELLER_INFO_COL = "seller_info"
SELLER_ID_COL  = "seller_id"
REVIEWS_COL    = "reviews"  

REVIEW_ID_COL_CANDIDATES = ["review_id", "id", "reviewId"]

# Thresholds (locked) 
MIN_WORDS = 5

ISSUE_SIM_THRESHOLD = 0.4
TOP_GAP_MIN = 0.05
TOP2_GAP_MAX = 0.04

NEG_PROB_THRESHOLD = 0.40

# Models
SIM_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
SENT_MODEL_NAME = "distilbert-base-uncased-finetuned-sst-2-english"

# Output tables
T_REVIEWS_BASE          = f"airbnb_reviews_base_{RUN_ID}"

T_SENTENCES_OCCUR       = f"airbnb_sentences_occurrences_{RUN_ID}"
T_SENTENCES_UNIQUE      = f"airbnb_sentences_unique_{RUN_ID}"

T_SIM_UNIQUE            = f"airbnb_similarity_unique_{RUN_ID}"

T_SENTIMENT_UNIQUE      = f"airbnb_sentiment_unique_{RUN_ID}"
T_NEG_UNIQUE            = f"airbnb_negative_unique_{RUN_ID}"
T_NEG_OCCUR             = f"airbnb_negative_occurrences_{RUN_ID}"

T_ISSUES_UNIQUE          = f"airbnb_issues_unique_{RUN_ID}"
T_ISSUES_OCCUR           = f"airbnb_issues_occurrences_{RUN_ID}"

T_PROP_ISSUE_AGG         = f"airbnb_property_issue_agg_{RUN_ID}"
T_HOST_ISSUE_AGG         = f"airbnb_host_issue_agg_{RUN_ID}"

T_PROP_ISSUE_EVID        = f"airbnb_property_issue_evidence_{RUN_ID}"
T_HOST_ISSUE_EVID        = f"airbnb_host_issue_evidence_{RUN_ID}"

T_ISSUES_CANDIDATES_UNIQUE = f"airbnb_issues_candidates_unique_{RUN_ID}"
T_ISSUES_CANDIDATES_OCCUR  = f"airbnb_issues_candidates_occurrences_{RUN_ID}"

T_SENTENCES_CANDIDATES_OCCUR  = f"airbnb_sentences_candidates_occurrences_{RUN_ID}"
T_SENTENCES_CANDIDATES_UNIQUE = f"airbnb_sentences_candidates_unique_{RUN_ID}"

T_SENT_PROGRESS = f"airbnb_sent_progress_{RUN_ID}"
T_SENT_INPUT    = f"airbnb_sent_input_{RUN_ID}"

print("[INFO] Output tables:")
for t in [
    T_REVIEWS_BASE, T_SENTENCES_OCCUR, T_SENTENCES_UNIQUE, T_SIM_UNIQUE,
    T_SENTIMENT_UNIQUE, T_NEG_UNIQUE, T_NEG_OCCUR, T_ISSUES_UNIQUE, T_ISSUES_OCCUR,
    T_PROP_ISSUE_AGG, T_HOST_ISSUE_AGG, T_PROP_ISSUE_EVID, T_HOST_ISSUE_EVID
]:
    print(" -", t)

[INFO] RUN_ID=ITALY_1
[INFO] Output tables:
 - airbnb_reviews_base_ITALY_1
 - airbnb_sentences_occurrences_ITALY_1
 - airbnb_sentences_unique_ITALY_1
 - airbnb_similarity_unique_ITALY_1
 - airbnb_sentiment_unique_ITALY_1
 - airbnb_negative_unique_ITALY_1
 - airbnb_negative_occurrences_ITALY_1
 - airbnb_issues_unique_ITALY_1
 - airbnb_issues_occurrences_ITALY_1
 - airbnb_property_issue_agg_ITALY_1
 - airbnb_host_issue_agg_ITALY_1
 - airbnb_property_issue_evidence_ITALY_1
 - airbnb_host_issue_evidence_ITALY_1


## Issues taxonomy (final approved)

In [0]:
ISSUE_TEXTS = {
    "wifi": (
        "internet connection quality, wifi speed, weak or unstable signal, "
        "slow internet, connection drops, unreliable wifi for work or streaming"
    ),
    "tv_and_entertainment": (
        "television, TV functionality, smart TV, streaming services, Netflix, "
        "channels, media system, remote control issues, entertainment setup"
    ),
    "workspace": (
        "workspace suitability, desk and chair, working remotely, laptop setup, "
        "comfortable space for work or study, home office conditions"
    ),
    "climate_control": (
        "air conditioning, heating system, temperature control, cooling or warmth, "
        "poor ventilation, too hot or too cold inside the property"
    ),
    "furniture_and_comfort": (
        "bed comfort, mattress quality, sofa or couch comfort, broken furniture, "
        "uncomfortable seating, sleeping quality, worn or damaged furniture"
    ),
    "cleanliness": (
        "cleanliness and hygiene, dirty or dusty apartment, unpleasant smells, "
        "stains, poor cleaning standards, lack of sanitation"
    ),
    "linens": (
        "bed linens, sheets, towels, blankets, linen cleanliness or freshness, "
        "old, dirty, or insufficient towels or bedding"
    ),
    "bathroom_and_water": (
        "bathroom condition, shower quality, toilet functionality, water pressure, "
        "hot water availability, plumbing problems, mold or mildew in bathroom"
    ),
    "noise": (
        "noise levels, loud neighbors, street noise, construction sounds, "
        "poor sound insulation, difficult to sleep due to noise"
    ),
    "space_and_privacy": (
        "apartment size, cramped or small space, layout and room arrangement, "
        "lack of privacy, shared spaces, inconvenient floor plan or room separation"
    ),
    "kitchen_and_appliances": (
        "kitchen facilities, cooking equipment, stove, oven, refrigerator, microwave, "
        "coffee machine, kettle, broken or missing kitchen appliances"
    ),
    "rules_and_access": (
        "check-in and check-out process, access to the property, keys or door codes, "
        "house rules, unclear instructions, restrictions, confusing entry or exit"
    ),
    "parking": (
        "parking availability, parking convenience, garage access, street parking, "
        "difficulty finding parking near the property"
    ),
    "accessibility": (
        "accessibility issues, elevator or lift availability, stairs only access, "
        "difficulty reaching the apartment, suitability for limited mobility"
    ),
    "location": (
        "location convenience, neighborhood quality, distance to attractions, "
        "public transportation access, safety or comfort of the surrounding area"
    ),
    "view": (
        "view from the property, balcony or window scenery, sea or city view, "
        "blocked or disappointing view compared to listing description"
    ),
    "safety": (
        "safety and security concerns, feeling unsafe, poor locks, building security, "
        "unsafe neighborhood or entrance"
    ),
    "property_condition": (
        "maintenance issues, broken items, poor condition, lack of repairs, "
        "lighting problems, insufficient storage space, overall property upkeep"
    ),
    "host_communication": (
        "communication with the host, slow or no response, lack of helpfulness, "
        "difficulty resolving problems, unresponsive or unclear communication"
    ),
    "pets": (
        "pet friendliness, pet restrictions, pet fees, issues with pets, "
        "unexpected rules regarding animals"
    ),
    "price_value": (
        "value for money, overpriced stay, price not matching quality, "
        "expectations versus cost, poor value compared to similar listings"
    ),
}
ISSUES = list(ISSUE_TEXTS.keys())
print("[INFO] #ISSUES =", len(ISSUES))

[INFO] #ISSUES = 21


## Step 0 - Base table: extract seller_id + drop nulls + (property_id, reviews) block-dedup

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType

df_base = df_italy

# Validate minimal inputs exist 
missing_inputs = [c for c in [PROPERTY_COL, REVIEWS_COL, SELLER_INFO_COL]if c not in df_base.columns]
if missing_inputs:
    raise Exception(f"[ERROR] Missing required columns in airbnb: {missing_inputs}\nFound: {df_base.columns}")

# Extract seller_id from seller_info JSON 
seller_schema = StructType([ StructField("seller_id", StringType(), True), StructField("url", StringType(), True), StructField("name", StringType(), True), ])
df_base = df_base.withColumn( "seller_parsed", F.from_json(F.col(SELLER_INFO_COL).cast("string"), seller_schema) ).withColumn( SELLER_ID_COL, F.col("seller_parsed.seller_id") ).drop("seller_parsed")
print("[OK] seller_id extracted from seller_info")

# Drop rows with missing critical keys/text
df_base = df_base.filter( F.col(PROPERTY_COL).isNotNull() & (F.length(F.col(PROPERTY_COL).cast("string")) > 0) & F.col(SELLER_ID_COL).isNotNull() & (F.length(F.col(SELLER_ID_COL).cast("string")) > 0) & F.col(REVIEWS_COL).isNotNull() & (F.length(F.trim(F.col(REVIEWS_COL).cast("string"))) > 0) )

# Keep only required cols downstream
df_base = df_base.select(PROPERTY_COL, SELLER_ID_COL, REVIEWS_COL)

# Basic quality checks (after filtering)
total_rows = df_base.count()
print("[INFO] rows (after dropping null keys/text):", total_rows)
print("[INFO] unique properties:",df_base.select(PROPERTY_COL).distinct().count())
print("[INFO] unique sellers:", df_base.select(SELLER_ID_COL).distinct().count())
print("[DEBUG] sample rows:")
display(df_base.limit(10))

# Save base table for the pipeline  
save_table_and_path(df_base, T_REVIEWS_BASE)
print("[INFO] base rows:", df_base.count())

[OK] seller_id extracted from seller_info
[INFO] rows (after dropping null keys/text): 90953
[INFO] unique properties: 85220
[INFO] unique sellers: 62704
[DEBUG] sample rows:


property_id seller_id reviews 53932806 435433356 ["Clean room and kind personnel. I had a really peaceful night and everything was spotless. The breakfast was also nice! The location is a bit random but still walking distance from both the center and the station.","Very nice room and bathroom. With included breakfast..Top Lake Garda 35 min away by car.","Super friendly staff attentive to the guest's needs. Easy to reach location from FS station and quite close to downtown, walking. Parking in front of the property for those arriving by car. Nearby also supermarket/shopping center is also open on Sundays. Have a great breakfast. I suggest the property for both shorter and longer stays.","I went to Rovereto for a business trip and slept one night in this beautiful Bed & Breakfast. Convenient parking under the property, nearby supermarket, and inside everything is perfect, clean modern equipped with all the necessary amenities. Breakfast was good and with a wide offer, Federico was very kind","Eco-conventional bed and breakfast that allows you to reach the city center also using the natural motor levers that we are composed of","Very comfortable drink, the staff is exceptional and always willing to meet you, highly recommended"] 1785172 9361088 ["Fabulous location, breathtaking views, wake up and have the sea! Very kind and helpful, big Massimo 🤙🏻","Everything was perfect beside Internet which was slow (3 Mbit per sec in my room) and unstable but ok for normal surfing and audio calls.","Very cozy and friendly shared accommodation with (kite) surfer Vibe. But even as a non-surfers, I felt very comfortable. Massimo is incredibly friendly and always helpful and gave me some good tips for day trips and activities. The room had a big sunny balcony. The communal terrace also invites you to sit. They were very nice days!","Massimo is an exceptional person, generous and welcoming! The place is enchanted, a short distance from the sea, an oasis of tranquility!","Nice location, quiet place.","A kite surfers paradise. Massimo was a super host. Everything about his wonderful home is amazing! I would definitely recommend a hundred times over.","Beautifully located cottage with many rooms and everything else you need. Massimo is a friendly and relaxed host who will help you with advice and deeds. The beach view to Gizzeria Lido is beautiful,but a vehicle is recommended to get there. Thx Yours sincerely, Steve","Massimo's place is a real dream if you want to experience the real South Italian slow life. Together with him and Gianna we had a super nice evening and it was a real bless staying over. I felt super unfortunate leaving the morning after, because this place is worth staying for a longer period than only one night.","great stay from Massimo, cozy spacious house with a nice sea view. Pleasant outdoor spaces for lunch in nature. Recommended","Great stay at Massimo's. Friendly, welcoming and reliable person. From the house there is a nice view of the sea. Recommended","This place in Gizzeria is like a little piece of heaven. Located on a hill overlooking the Bay with amazing views every moment. Just 5 mins drive to the beach. You can get fresh fruits from the trees around here :) and the best olive oil ever. Massimo is a very friendly, open minded and extremely helpful person. This place is a bit secluded so you need a car to get around which i didn't have. But Massimo helped me get around, go to the beach or groceries. He also picked me up at the airport. I really have nothing to complain and highly recommend this place to other tourists. The place itself is very comfortable to stay for a short or a long visit. I stayed here for a month. And i will definitely come back for more :D","The place has a stunning view and is definitely worth stopping at for a while. The beach is 5 minutes away by car, and Massimo himself is an amazing kite and windsurf instructor with years of stories. On top of that, the hosts are warm and welcoming, and we sha

[OK] Saved table=airbnb_reviews_base_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_reviews_base_ITALY_1
[INFO] base rows: 90953


In [0]:
# Remove duplicated reviews inside the same property before sentence splitting.
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType
import pandas as pd
import re
import pandas as pd

# Output table for this step
T_REVIEW_CHUNKS_BASE = f"airbnb_review_chunks_base_{RUN_ID}"
print("[INFO] Output:", T_REVIEW_CHUNKS_BASE)

df0 = spark.table(T_REVIEWS_BASE)

# Basic checks
print("[INFO] T_REVIEWS_BASE rows:", df0.count())
display(df0.select(PROPERTY_COL, SELLER_ID_COL, REVIEWS_COL).limit(5))

# Schema for occurrences (before dedup)
chunks_schema = StructType([
    StructField(PROPERTY_COL, StringType(), True),
    StructField(SELLER_ID_COL, StringType(), True),
    StructField("review_chunk", StringType(), True),
])

def split_reviews_to_chunks(pdf_iter):
    """
    Split the 'reviews' blob into individual reviews FIRST.
    Primary rule: split on the delimiter  ","  (typical for scraped list-of-quotes reviews).
    Fallback: paragraph/newline/bullets split.
    """
    # Detect "quoted reviews list" delimiter:  ","  (allow whitespace/newlines between)
    # Example: ...hotspots.","A perfect stay...<br/>Straight down...
    review_delim_re = re.compile(r'"\s*,\s*"')

    # Fallback paragraph-ish split (kept from your original logic, slightly simplified)
    fallback_split_re = re.compile(r"(?:\n\s*\n)+|(?:\n[\-\*\u2022]\s+)|(?:\n{1,})")

    # HTML <br/> -> newline
    br_re = re.compile(r"<br\s*/?>", flags=re.IGNORECASE)

    # Strip outer wrappers like ["...","..."] or leading/trailing quotes
    strip_wrappers_re = re.compile(r'^\s*\[\s*"|"\s*\]\s*$')

    for pdf in pdf_iter:
        props = pdf[PROPERTY_COL].astype(str).tolist()
        sellers = pdf[SELLER_ID_COL].astype(str).tolist()
        blobs = pdf[REVIEWS_COL].fillna("").astype(str).tolist()

        out = []
        for i, blob in enumerate(blobs):
            t = (blob or "").strip()
            if not t:
                continue

            # normalize HTML breaks
            t = br_re.sub("\n", t)

            # If it looks like a quoted-list blob, split by "," between quotes
            parts = []
            if '","' in t or review_delim_re.search(t):
                # remove outer wrappers if exist
                t2 = strip_wrappers_re.sub("", t)
                # split by delimiter
                raw_parts = review_delim_re.split(t2)

                # clean each review: strip stray quotes/commas/spaces
                for p in raw_parts:
                    p = (p or "").strip()
                    p = p.strip(' "')
                    p = p.strip()
                    if p:
                        parts.append(p)

            # Fallback: if delimiter logic didn’t produce anything
            if not parts:
                parts = [p.strip() for p in fallback_split_re.split(t) if p and p.strip()]
                if not parts:
                    parts = [t]

            for ch in parts:
                ch = re.sub(r"\s+", " ", ch).strip()
                if len(ch) < 20:
                    continue
                out.append({
                    PROPERTY_COL: props[i],
                    SELLER_ID_COL: sellers[i],
                    "review_chunk": ch
                })

        yield pd.DataFrame(out, columns=[PROPERTY_COL, SELLER_ID_COL, "review_chunk"])

# 1) Split to chunk occurrences
df_chunks_occ = (
    df0.select(PROPERTY_COL, SELLER_ID_COL, REVIEWS_COL)
       .repartition(400, SELLER_ID_COL)
       .mapInPandas(split_reviews_to_chunks, schema=chunks_schema)
)

rows_chunks_occ = df_chunks_occ.count()
print(f"[INFO] review_chunk occurrences (before dedup): {rows_chunks_occ:,}")
display(df_chunks_occ.select(F.length("review_chunk").alias("len")).summary("min","25%","50%","mean","75%","max"))

# 2) Normalize + dedup per (property_id, review_chunk_norm)
df_chunks_occ = df_chunks_occ.withColumn("review_chunk", F.trim(F.col("review_chunk")))
df_chunks_occ = df_chunks_occ.withColumn("review_chunk", F.regexp_replace(F.col("review_chunk"), r"\s+", " "))

df_chunks_occ = df_chunks_occ.withColumn("review_chunk_norm", F.lower(F.col("review_chunk")))
df_chunks_occ = df_chunks_occ.withColumn("review_chunk_norm", F.regexp_replace(F.col("review_chunk_norm"), r"\s+", " "))
df_chunks_occ = df_chunks_occ.withColumn("review_chunk_norm", F.trim(F.col("review_chunk_norm")))

# Apply dedup
df_chunks_unique = df_chunks_occ.dropDuplicates([PROPERTY_COL, "review_chunk_norm"])

rows_chunks_unique = df_chunks_unique.count()
removed = rows_chunks_occ - rows_chunks_unique
print(f"[OK] Chunk dedup done: unique={rows_chunks_unique:,} | removed={removed:,} ({(removed/rows_chunks_occ):.2%} saved)")

# QA: show properties with most removed chunks
prop_before = df_chunks_occ.groupBy(PROPERTY_COL).count().withColumnRenamed("count", "chunks_before")
prop_after  = df_chunks_unique.groupBy(PROPERTY_COL).count().withColumnRenamed("count", "chunks_after")

prop_delta = (
    prop_before.join(prop_after, on=PROPERTY_COL, how="inner")
               .withColumn("removed_chunks", F.col("chunks_before") - F.col("chunks_after"))
               .orderBy(F.desc("removed_chunks"))
)

print("[DEBUG] Properties with most duplicated chunks removed:")
display(prop_delta.limit(30))

# 3) Add stable ids for downstream joins / traceability
df_chunks_unique = df_chunks_unique.withColumn("review_chunk_id", F.sha2(F.col("review_chunk_norm"), 256))

# Keep only needed cols
df_chunks_unique = df_chunks_unique.select(
    PROPERTY_COL, SELLER_ID_COL,
    "review_chunk_id",
    "review_chunk",
    "review_chunk_norm"
)

# 4) Save
save_table_and_path(df_chunks_unique, T_REVIEW_CHUNKS_BASE)
print("[INFO] review_chunks unique:", df_chunks_unique.count())


[INFO] Output: airbnb_review_chunks_base_ITALY_1
[INFO] T_REVIEWS_BASE rows: 90953


property_id seller_id reviews 53932806 435433356 ["Clean room and kind personnel. I had a really peaceful night and everything was spotless. The breakfast was also nice! The location is a bit random but still walking distance from both the center and the station.","Very nice room and bathroom. With included breakfast..Top Lake Garda 35 min away by car.","Super friendly staff attentive to the guest's needs. Easy to reach location from FS station and quite close to downtown, walking. Parking in front of the property for those arriving by car. Nearby also supermarket/shopping center is also open on Sundays. Have a great breakfast. I suggest the property for both shorter and longer stays.","I went to Rovereto for a business trip and slept one night in this beautiful Bed & Breakfast. Convenient parking under the property, nearby supermarket, and inside everything is perfect, clean modern equipped with all the necessary amenities. Breakfast was good and with a wide offer, Federico was very kind","Eco-conventional bed and breakfast that allows you to reach the city center also using the natural motor levers that we are composed of","Very comfortable drink, the staff is exceptional and always willing to meet you, highly recommended"] 1785172 9361088 ["Fabulous location, breathtaking views, wake up and have the sea! Very kind and helpful, big Massimo 🤙🏻","Everything was perfect beside Internet which was slow (3 Mbit per sec in my room) and unstable but ok for normal surfing and audio calls.","Very cozy and friendly shared accommodation with (kite) surfer Vibe. But even as a non-surfers, I felt very comfortable. Massimo is incredibly friendly and always helpful and gave me some good tips for day trips and activities. The room had a big sunny balcony. The communal terrace also invites you to sit. They were very nice days!","Massimo is an exceptional person, generous and welcoming! The place is enchanted, a short distance from the sea, an oasis of tranquility!","Nice location, quiet place.","A kite surfers paradise. Massimo was a super host. Everything about his wonderful home is amazing! I would definitely recommend a hundred times over.","Beautifully located cottage with many rooms and everything else you need. Massimo is a friendly and relaxed host who will help you with advice and deeds. The beach view to Gizzeria Lido is beautiful,but a vehicle is recommended to get there. Thx Yours sincerely, Steve","Massimo's place is a real dream if you want to experience the real South Italian slow life. Together with him and Gianna we had a super nice evening and it was a real bless staying over. I felt super unfortunate leaving the morning after, because this place is worth staying for a longer period than only one night.","great stay from Massimo, cozy spacious house with a nice sea view. Pleasant outdoor spaces for lunch in nature. Recommended","Great stay at Massimo's. Friendly, welcoming and reliable person. From the house there is a nice view of the sea. Recommended","This place in Gizzeria is like a little piece of heaven. Located on a hill overlooking the Bay with amazing views every moment. Just 5 mins drive to the beach. You can get fresh fruits from the trees around here :) and the best olive oil ever. Massimo is a very friendly, open minded and extremely helpful person. This place is a bit secluded so you need a car to get around which i didn't have. But Massimo helped me get around, go to the beach or groceries. He also picked me up at the airport. I really have nothing to complain and highly recommend this place to other tourists. The place itself is very comfortable to stay for a short or a long visit. I stayed here for a month. And i will definitely come back for more :D","The place has a stunning view and is definitely worth stopping at for a while. The beach is 5 minutes away by car, and Massimo himself is an amazing kite and windsurf instructor with years of stories. On top of that, the hosts are warm and welcoming, and we sha

[INFO] review_chunk occurrences (before dedup): 1,243,638


summary,len
min,20
25%,127
50%,232
mean,287.60315541982476
75%,381
max,7590


[OK] Chunk dedup done: unique=1,172,573 | removed=71,065 (5.71% saved)
[DEBUG] Properties with most duplicated chunks removed:


property_id,chunks_before,chunks_after,removed_chunks
585240065278294204,118,36,82
24752585,96,25,71
53524651,92,24,68
17446225,96,29,67
301842,96,30,66
43649599,96,37,59
2922931,96,37,59
6152585,96,39,57
539013896238702081,96,39,57
1091908249105854120,93,41,52


[OK] Saved table=airbnb_review_chunks_base_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_review_chunks_base_ITALY_1
[INFO] review_chunks unique: 1172573


In [0]:
from pyspark.sql import functions as F

print("[QA] Example raw reviews blob (first row):")
display(df0.select(REVIEWS_COL).limit(1))

print("[QA] Sample produced review_chunks:")
display(df_chunks_occ.select(PROPERTY_COL, "review_chunk", F.length("review_chunk").alias("len")).limit(10))

print("[QA] How many chunks per property (top 20):")
display(
    df_chunks_occ.groupBy(PROPERTY_COL)
                 .agg(F.count("*").alias("n_chunks"))
                 .orderBy(F.desc("n_chunks"))
                 .limit(20)
)


[QA] Example raw reviews blob (first row):


reviews
"[""Clean room and kind personnel. I had a really peaceful night and everything was spotless. The breakfast was also nice! The location is a bit random but still walking distance from both the center and the station."",""Very nice room and bathroom. With included breakfast..Top Lake Garda 35 min away by car."",""Super friendly staff attentive to the guest's needs. Easy to reach location from FS station and quite close to downtown, walking. Parking in front of the property for those arriving by car.Nearby also supermarket/shopping center is also open on Sundays.Have a great breakfast.I suggest the property for both shorter and longer stays."",""I went to Rovereto for a business trip and slept one night in this beautiful Bed & Breakfast. Convenient parking under the property, nearby supermarket, and inside everything is perfect, clean modern equipped with all the necessary amenities. Breakfast was good and with a wide offer, Federico was very kind"",""Eco-conventional bed and breakfast that allows you to reach the city center also using the natural motor levers that we are composed of"",""Very comfortable drink, the staff is exceptional and always willing to meet you, highly recommended""]"


[QA] Sample produced review_chunks:


property_id,review_chunk,len
74463,"We were kindly welcomed by Carlo, who instructed us by phone to check in. He was extremely flexible with our arrival time and everything worked out wonderfully. If you have any questions or problems, he was always available and helped us. The apartment is nicely located next to the park overlooking the treetops from the large balcony. In the apartment, everything is available, which is necessary for an extended stay. Unfortunately, the pool was not yet open in May, but the sea is also not far on foot. Similarly, you can reach the bus and metromare in about 5 minutes and get everywhere from here. Everything was a wonderful fit for us. Gladly again",654
74463,"r the pool, On the other side is a beautiful park with sports and playgrounds. The area is very quiet and quiet. There is a good grocery store nearby. For us it was not far to go to the sea. Thank you very much for a wonderful holiday. We would like to return here again.",271
74463,This was perfect for me! I was competing at the Stade Nuoto in the Italo Nicoletti centre and this was walking distance for me. Everything was close at hand with many shops and restaurants to choose from. The street was quiet and I was comfortable.,248
27453632,"An incredible stay, what can I say. Located in a wonderful village, and the view from the room was simply breathtaking. Irena and her husband were very friendly and responsive. Highly recommended as a base to visit the Langhe.",226
27453632,Marvellous location due to outstanding views and wonderful little town,70
27453632,Wonderful guest apartment that is stylishly decorated and amazing views! The location is perfect - 3 minutes walk from town center. Irene and husband Daniel are very hospitable and politely shared local tips with me. They are easy to communicate with and very nice and helpful. Very special place!,297
27453632,"Idyllic stay in Monforte at Irene's! Her place is charming, spacious, tastefully remodeled, and very comfortable. But Irene and her husband are mostly charming hosts and very welcoming, and accommodating. The gem is of course the stunning views of the area from the balcony of the accommodation! Would highly recommend staying at Irene's place and visiting the area",365
27453632,"Super nice people. Huge private room with bathroom. Beautiful view as U r high in hill, and many nice lunch/diner places around. Good WIFI. And simple but good breakfast included. 5 Stars !!!!",192
27453632,We enjoyed our stay very much! The room had an amazing view over the yineyards and the close-by villages and the interior was really beautiful. Irene and her husband where very friendly and helpful in every possible way. I can‘t think of a better way to spend some time in this region.,285
27453632,"Very very beautiful, both inside and out. The patio has a gorgeous view of a valley covered with vineyards. The room itself is very stylish and spacious. The location was nice. The hosts were nice. Everything was nice.",218


[QA] How many chunks per property (top 20):


property_id,n_chunks
585240065278294204,118
33669391,96
6152585,96
17446225,96
43649599,96
301842,96
24752585,96
539013896238702081,96
2922931,96
54363887,95


## Step 1 - Sentence splitting + English-only + min_words + smart normalization (for dedup)
**Outputs:**
- `T_SENTENCES_OCCUR`: all sentence occurrences (keeps property/host linkage)
- `T_SENTENCES_UNIQUE`: unique sentences by `sentence_norm` for model inference

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType
import pandas as pd

TEXT_COL = "review_chunk"

occur_schema = StructType([
    StructField(PROPERTY_COL, StringType(), True),
    StructField(SELLER_ID_COL, StringType(), True),
    StructField("review_chunk_id", StringType(), True),
    StructField("sentence", StringType(), True),
])

def split_to_sentences(pdf_iter):
    import re
    splitter = re.compile(r"(?<=[\.\!\?])\s+|\n+")

    for pdf in pdf_iter:
        out_rows = []
        prop   = pdf[PROPERTY_COL].astype(str).tolist()
        seller = pdf[SELLER_ID_COL].astype(str).tolist()
        cid    = pdf["review_chunk_id"].astype(str).tolist()
        texts  = pdf[TEXT_COL].fillna("").astype(str).tolist()

        for i, text in enumerate(texts):
            t = (text or "").strip()
            if not t:
                continue

            parts = [p.strip() for p in splitter.split(t) if p and p.strip()]
            for s in parts:
                out_rows.append({
                    PROPERTY_COL: prop[i],
                    SELLER_ID_COL: seller[i],
                    "review_chunk_id": cid[i],
                    "sentence": s
                })

        yield pd.DataFrame(out_rows, columns=[PROPERTY_COL, SELLER_ID_COL, "review_chunk_id", "sentence"])


df_in = spark.table(T_REVIEW_CHUNKS_BASE)

df_occ = (
    df_in
    .select(PROPERTY_COL, SELLER_ID_COL, "review_chunk_id", F.col(TEXT_COL).alias(TEXT_COL))
    .repartition(400, SELLER_ID_COL)
    .mapInPandas(split_to_sentences, schema=occur_schema)
)

# clean
df_occ = df_occ.withColumn("sentence", F.trim(F.col("sentence")))
df_occ = df_occ.withColumn("sentence", F.regexp_replace(F.col("sentence"), r"\s+", " "))

# min_words
df_occ = df_occ.withColumn("n_words", F.size(F.split(F.col("sentence"), r"\s+")))
df_occ = df_occ.filter(F.col("n_words") >= F.lit(int(MIN_WORDS)))

# english-only heuristic (strict ASCII + some latin letters)
df_occ = df_occ.withColumn("non_ascii", F.length(F.regexp_replace(F.col("sentence"), r"[\x00-\x7F]", "")))
df_occ = df_occ.withColumn("latin_letters", F.length(F.regexp_replace(F.col("sentence"), r"[^A-Za-z]", "")))
df_occ = df_occ.filter((F.col("non_ascii") <= F.lit(0)) & (F.col("latin_letters") >= F.lit(3)))

# norm for smart dedup
df_occ = df_occ.withColumn("sentence_norm", F.lower(F.col("sentence")))
df_occ = df_occ.withColumn("sentence_norm", F.regexp_replace(F.col("sentence_norm"), r"\s+", " "))
df_occ = df_occ.withColumn("sentence_norm", F.regexp_replace(F.col("sentence_norm"), r"^[\.\,\!\?\:\;\-]+", ""))
df_occ = df_occ.withColumn("sentence_norm", F.regexp_replace(F.col("sentence_norm"), r"[\.\,\!\?\:\;\-]+$", ""))
df_occ = df_occ.withColumn("sentence_norm", F.trim(F.col("sentence_norm")))

# stable id
df_occ = df_occ.withColumn("sentence_id", F.sha2(F.col("sentence_norm"), 256))

save_table_and_path(
    df_occ.select(PROPERTY_COL, SELLER_ID_COL, "review_chunk_id",
                  "sentence_id", "sentence", "sentence_norm", "n_words"),
    T_SENTENCES_OCCUR
)

df_unique = (
    spark.table(T_SENTENCES_OCCUR)
         .select("sentence_id", "sentence", "sentence_norm")
         .dropDuplicates(["sentence_norm"])
)
save_table_and_path(df_unique, T_SENTENCES_UNIQUE)

print("[INFO] sentences_occur:", spark.table(T_SENTENCES_OCCUR).count())
print("[INFO] sentences_unique:", spark.table(T_SENTENCES_UNIQUE).count())


[OK] Saved table=airbnb_sentences_occurrences_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_sentences_occurrences_ITALY_1
[OK] Saved table=airbnb_sentences_unique_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_sentences_unique_ITALY_1
[INFO] sentences_occur: 3965611
[INFO] sentences_unique: 3665685


## Step 2 - Candidate Sentence Prefilter (Top-K)
Selects a small, high-signal subset of review sentences per property for downstream semantic matching and API processing.


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Params
K_HI = 25
K_FALLBACK = 15
TARGET_MAX = K_HI + K_FALLBACK

# Load occurrences
df_occ = spark.table(T_SENTENCES_OCCUR).select(
    PROPERTY_COL, SELLER_ID_COL, "sentence_id", "sentence", "sentence_norm", "n_words"
)

# High-signal heuristic 
HIGH_SIGNAL_TERMS = [
    # negation / mismatch
    "not", "no", "never", "without", "can't", "cannot", "didn't", "doesn't", "wasn't", "weren't",
    "as described", "not as", "different than", "nothing like", "misleading", "false advertising",

    # complaints / dissatisfaction
    "complaint", "complain", "disappointed", "disappointing", "unhappy", "regret", "avoid",
    "terrible", "awful", "horrible", "worst", "bad", "poor", "mediocre", "unacceptable",

    # service / communication
    "rude", "unresponsive", "ignored", "no response", "didn't respond", "host didn't",
    "late", "delay", "delayed", "cancelled", "canceled",

    # functionality / maintenance
    "broken", "not working", "doesn't work", "didn't work", "failed", "issue", "problem",
    "malfunction", "leak", "leaking", "clogged", "blocked", "stuck", "damaged",

    # cleanliness / hygiene
    "dirty", "filthy", "unclean", "smell", "smelly", "odor", "stains", "mold", "mildew",
    "bugs", "cockroach", "roaches", "insects", "ants", "bedbug", "bed bugs", "hair",

    # noise / safety
    "noisy", "loud", "noise", "unsafe", "dangerous", "scary",

    # value / pricing
    "overpriced", "not worth", "waste", "ripoff", "scam", "expensive",

    # comfort / sleep / climate
    "uncomfortable", "hard bed", "bed hard", "small", "tiny", "cramped",
    "cold", "freezing", "hot", "heat", "ac", "air conditioning", "air conditioner", "no ac",
    "heater", "no heat", "no hot water", "hot water",
]

def _mk_or_regex(terms):
    import re
    return "(" + "|".join(re.escape(t) for t in terms) + ")"

df = (
    df_occ
    .withColumn("sent_lc", F.col("sentence_norm"))
    .withColumn("is_high_signal", F.col("sent_lc").rlike(_mk_or_regex(HIGH_SIGNAL_TERMS)))
)

# pick HI per property
w_hi = Window.partitionBy(PROPERTY_COL).orderBy(F.desc("is_high_signal"), F.desc("n_words"))
pick_hi = (
    df.filter(F.col("is_high_signal") == True)
      .withColumn("rk", F.row_number().over(w_hi))
      .filter(F.col("rk") <= F.lit(int(K_HI)))
      .drop("rk")
)

# fallback: fill remaining budget by n_words
seed_keys = pick_hi.select(PROPERTY_COL, "sentence_norm").dropDuplicates()
seed_counts = pick_hi.groupBy(PROPERTY_COL).agg(F.count("*").alias("seed_n"))

df_fb = (
    df.join(seed_counts, on=PROPERTY_COL, how="left").fillna({"seed_n": 0})
      .join(seed_keys, on=[PROPERTY_COL, "sentence_norm"], how="left_anti")
      .withColumn(
          "need_n",
          F.when(F.col("seed_n") >= F.lit(TARGET_MAX), F.lit(0))
           .otherwise(F.lit(TARGET_MAX) - F.col("seed_n"))
      )
      .withColumn("need_n", F.least(F.col("need_n"), F.lit(int(K_FALLBACK))))
)

w_fb = Window.partitionBy(PROPERTY_COL).orderBy(F.desc("n_words"))
pick_fb = (
    df_fb.withColumn("rk", F.row_number().over(w_fb))
         .filter(F.col("rk") <= F.col("need_n"))
         .drop("rk", "need_n", "seed_n")
)

# Final candidates + save
df_cand = pick_hi.unionByName(pick_fb).dropDuplicates([PROPERTY_COL, "sentence_norm"])
save_table_and_path(df_cand, T_SENTENCES_CANDIDATES_OCCUR)

df_cand_unique = (
    df_cand.select("sentence_id", "sentence", "sentence_norm")
           .dropDuplicates(["sentence_norm"])
)
save_table_and_path(df_cand_unique, T_SENTENCES_CANDIDATES_UNIQUE)

print("[INFO] prefilter in:", df_occ.count(), "| out:", df_cand.count())
print("[INFO] prefilter unique:", df_cand_unique.count())


[OK] Saved table=airbnb_sentences_candidates_occurrences_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_sentences_candidates_occurrences_ITALY_1
[OK] Saved table=airbnb_sentences_candidates_unique_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_sentences_candidates_unique_ITALY_1
[INFO] prefilter in: 3965611 | out: 2277365
[INFO] prefilter unique: 2240196


## Step 3 – Semantic Similarity-Based Issue Matching
We compute Top1/Top2 + gap

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, FloatType

sim_schema = StructType([
    StructField("sentence_id", StringType(), True),
    StructField("sentence", StringType(), True),
    StructField("sentence_norm", StringType(), True),

    StructField("top1_issue", StringType(), True),
    StructField("top1_sim", FloatType(), True),
    StructField("top2_issue", StringType(), True),
    StructField("top2_sim", FloatType(), True),
    StructField("top_gap", FloatType(), True),

    StructField("sim_model", StringType(), True),
])

def make_similarity_once_fn_fast(model_name: str, issue_texts: dict, batch_size: int = 256, issue_batch: int = 64):
    def _fn(pdf_iter):
        import numpy as np
        import torch
        from functools import lru_cache

        # Prevent CPU thread explosion per executor
        torch.set_num_threads(1)

        @lru_cache(maxsize=1)
        def get_model_and_issue_emb():
            from sentence_transformers import SentenceTransformer
            # force cpu (since you don't have cuda currently)
            model = SentenceTransformer(model_name, device="cpu")
            issues = list(issue_texts.keys())
            issue_desc = [issue_texts[k] for k in issues]
            issue_emb = model.encode(
                issue_desc,
                normalize_embeddings=True,
                batch_size=issue_batch,
                show_progress_bar=False,
            )
            return model, issues, np.asarray(issue_emb, dtype=np.float32)

        model, issues, issue_emb = get_model_and_issue_emb()

        for pdf in pdf_iter:
            sents = pdf["sentence"].fillna("").astype(str).tolist()

            emb = model.encode(
                sents,
                normalize_embeddings=True,
                batch_size=batch_size,
                show_progress_bar=False,
            )
            emb = np.asarray(emb, dtype=np.float32)

            sims = emb @ issue_emb.T  # cosine similarity because normalized

            # Fast top-2 (no full sort)
            top2 = np.argpartition(-sims, kth=1, axis=1)[:, :2]
            row = np.arange(sims.shape[0])
            a = top2[:, 0]
            b = top2[:, 1]
            a_better = sims[row, a] >= sims[row, b]
            top1_i = np.where(a_better, a, b)
            top2_i = np.where(a_better, b, a)

            top1_sim = sims[row, top1_i].astype(np.float32)
            top2_sim = sims[row, top2_i].astype(np.float32)
            top_gap = (top1_sim - top2_sim).astype(np.float32)

            out = pdf[["sentence_id", "sentence", "sentence_norm"]].copy()
            out["top1_issue"] = [issues[i] for i in top1_i]
            out["top1_sim"] = top1_sim
            out["top2_issue"] = [issues[i] for i in top2_i]
            out["top2_sim"] = top2_sim
            out["top_gap"] = top_gap
            out["sim_model"] = "minilm"
            yield out
    return _fn

df_unique = spark.table(T_SENTENCES_CANDIDATES_UNIQUE)

# choose partitions sanely for CPU
N = max(32, min(128, spark.sparkContext.defaultParallelism * 2))
print("[INFO] Similarity partitions:", N)

fn_sim_once = make_similarity_once_fn_fast(SIM_MODEL_NAME, ISSUE_TEXTS, batch_size=128)

df_sim = (
    df_unique
    .repartition(N, "sentence_id")
    .mapInPandas(fn_sim_once, schema=sim_schema)
)

save_table_and_path(df_sim, T_SIM_UNIQUE)
print("[INFO] sim rows:", spark.table(T_SIM_UNIQUE).count())

display(spark.table(T_SIM_UNIQUE)
       .select("top1_sim","top2_sim","top_gap")
       .sample(0.001, seed=42)
       .summary("min","50%","mean","max"))

[INFO] Similarity partitions: 32
[INFO] sim rows: 2240196


summary,top1_sim,top2_sim,top_gap
min,0.018720116,-0.0023442581,3.9979815E-5
50%,0.3598643,0.27679205,0.055841193
mean,0.35208696678716256,0.2806541600107373,0.07143280679819645
max,0.7212724,0.5560351,0.45468774


## Apply similarity thresholds (Top1 + optional Top2) BEFORE sentiment
We define candidate issue tags from similarity only.

**Rules:**
- Keep Top1 if `top1_sim >= ISSUE_SIM_THRESHOLD`
- Add Top2 only if `top2_sim >= ISSUE_SIM_THRESHOLD` AND `top_gap <= TOP2_GAP_MAX`
- `TOP_GAP_MIN` is QA signal (we keep it for analysis, not for hard drop)

In [0]:
df_sim = spark.table(T_SIM_UNIQUE)

# Top1 candidates
df_top1 = (
    df_sim
    .filter(F.col("top1_sim") >= F.lit(float(ISSUE_SIM_THRESHOLD)))
    .select(
        "sentence_id","sentence","sentence_norm",
        "top1_issue","top1_sim","top2_issue","top2_sim","top_gap","sim_model",
        F.col("top1_issue").alias("issue"),
        F.col("top1_sim").alias("issue_sim"),
        F.lit(False).alias("is_top2")
    )
)

# Top2 candidates (only when very close)
df_top2 = (
    df_sim
    .filter((F.col("top2_sim") >= F.lit(float(ISSUE_SIM_THRESHOLD))) & (F.col("top_gap") <= F.lit(float(TOP2_GAP_MAX))))
    .select(
        "sentence_id","sentence","sentence_norm",
        "top1_issue","top1_sim","top2_issue","top2_sim","top_gap","sim_model",
        F.col("top2_issue").alias("issue"),
        F.col("top2_sim").alias("issue_sim"),
        F.lit(True).alias("is_top2")
    )
)

df_issues_candidates = df_top1.unionByName(df_top2)

save_table_and_path(df_issues_candidates, T_ISSUES_CANDIDATES_UNIQUE)
print("[INFO] issue-candidate rows:", spark.table(T_ISSUES_CANDIDATES_UNIQUE).count())

icu = spark.table(T_ISSUES_CANDIDATES_UNIQUE).cache()
display(icu.groupBy("issue").count().orderBy(F.desc("count")).limit(30))


[OK] Saved table=airbnb_issues_candidates_unique_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_issues_candidates_unique_ITALY_1
[INFO] issue-candidate rows: 991774


issue,count
space_and_privacy,303247
location,299042
noise,43425
view,42711
cleanliness,41575
parking,35909
kitchen_and_appliances,30652
workspace,27894
furniture_and_comfort,25006
host_communication,23807


## Step 4 - Sentiment (SST-2) ONLY on the similarity-candidates
Keep where `neg_prob >= 0.40`.

In [0]:
from pyspark.sql import functions as F

N_SENT_SHARDS = 40
T_SENT_INPUT = f"airbnb_sent_input_{RUN_ID}"
T_SENT_PROGRESS = f"airbnb_sent_progress_{RUN_ID}"

cand_issues = spark.table(T_ISSUES_CANDIDATES_UNIQUE).select("sentence_id","sentence").dropDuplicates(["sentence_id"])
sent_input = (
    cand_issues
    .withColumn("sent_shard", F.pmod(F.hash(F.col("sentence_id")), F.lit(int(N_SENT_SHARDS))))
)

sent_input.write.mode("overwrite").format("delta").saveAsTable(T_SENT_INPUT)
print("[OK] Saved:", T_SENT_INPUT, "| rows:", spark.table(T_SENT_INPUT).count())

# progress table (append-only)
progress_schema = "sent_shard INT, ts STRING, rows_scored LONG"
if not spark.catalog.tableExists(T_SENT_PROGRESS):
    spark.createDataFrame([], progress_schema).write.mode("overwrite").format("delta").saveAsTable(T_SENT_PROGRESS)
    print("[OK] Init progress table:", T_SENT_PROGRESS)
else:
    print("[OK] Progress exists (resume):", T_SENT_PROGRESS, "| rows:", spark.table(T_SENT_PROGRESS).count())
print("[OK] Init progress table:", T_SENT_PROGRESS)

display(spark.table(T_SENT_INPUT).groupBy("sent_shard").count().orderBy(F.desc("count")).limit(10))

[OK] Saved: airbnb_sent_input_ITALY_1 | rows: 847606
[OK] Init progress table: airbnb_sent_progress_ITALY_1
[OK] Init progress table: airbnb_sent_progress_ITALY_1


sent_shard,count
23,21499
10,21403
9,21383
19,21354
36,21337
16,21334
0,21306
28,21303
31,21301
39,21300


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, FloatType
import pandas as pd
from datetime import datetime

sent_schema = StructType([
    StructField("sentence_id", StringType(), True),
    StructField("neg_prob", FloatType(), True),
    StructField("pos_prob", FloatType(), True),
    StructField("sentiment_label", StringType(), True),
    StructField("sent_model", StringType(), True),
])

progress_schema = "sent_shard INT, ts STRING, rows_scored LONG"

def make_sst2_fn(
    model_name: str,
    batch_size_cpu: int = 64,
    batch_size_gpu: int = 256,
    max_len: int = 128,
):
    def _fn(pdf_iter):
        import os
        import numpy as np
        import torch
        from transformers import AutoTokenizer, AutoModelForSequenceClassification
        from functools import lru_cache

        # reduce overhead / thread explosion
        os.environ["TOKENIZERS_PARALLELISM"] = "false"
        torch.set_num_threads(1)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        bs = batch_size_gpu if device.type == "cuda" else batch_size_cpu

        @lru_cache(maxsize=1)
        def get_tok_model():
            tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
            mdl = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
            mdl.eval()
            return tok, mdl

        tok, mdl = get_tok_model()
        labels = ["negative", "positive"]

        use_amp = (device.type == "cuda")
        autocast_ctx = torch.cuda.amp.autocast if use_amp else None

        for pdf in pdf_iter:
            if pdf is None or pdf.empty:
                yield pd.DataFrame(columns=["sentence_id","neg_prob","pos_prob","sentiment_label","sent_model"])
                continue

            texts = pdf["sentence"].fillna("").astype(str).tolist()
            ids   = pdf["sentence_id"].astype(str).tolist()
            n = len(texts)

            neg = np.zeros(n, dtype=np.float32)
            pos = np.zeros(n, dtype=np.float32)
            lab = np.empty(n, dtype=object)

            # faster than no_grad for inference
            with torch.inference_mode():
                for start in range(0, n, bs):
                    end = min(n, start + bs)
                    enc = tok(
                        texts[start:end],
                        padding=True,
                        truncation=True,
                        max_length=max_len,
                        return_tensors="pt"
                    )
                    enc = {k: v.to(device) for k, v in enc.items()}

                    if use_amp:
                        with autocast_ctx():
                            logits = mdl(**enc).logits
                    else:
                        logits = mdl(**enc).logits

                    probs = torch.softmax(logits, dim=-1).float().cpu().numpy()
                    neg[start:end] = probs[:, 0]
                    pos[start:end] = probs[:, 1]
                    lab[start:end] = [labels[i] for i in probs.argmax(axis=1)]

            yield pd.DataFrame({
                "sentence_id": ids,
                "neg_prob": neg,
                "pos_prob": pos,
                "sentiment_label": lab,
                "sent_model": ["sst2"] * n
            })
    return _fn


# Init output/progress only if missing (resume-safe)
if not spark.catalog.tableExists(T_SENTIMENT_UNIQUE):
    spark.createDataFrame([], sent_schema).write.mode("overwrite").format("delta").saveAsTable(T_SENTIMENT_UNIQUE)
    print("[OK] Init output:", T_SENTIMENT_UNIQUE)
else:
    print("[OK] Output exists (resume):", T_SENTIMENT_UNIQUE)

if not spark.catalog.tableExists(T_SENT_PROGRESS):
    spark.createDataFrame([], progress_schema).write.mode("overwrite").format("delta").saveAsTable(T_SENT_PROGRESS)
    print("[OK] Init progress:", T_SENT_PROGRESS)
else:
    print("[OK] Progress exists (resume):", T_SENT_PROGRESS)


fn_sst2 = make_sst2_fn(SENT_MODEL_NAME)

# Partitions: keep small to reduce overhead per shard 
P = max(1, min(8, spark.sparkContext.defaultParallelism * 2))
print("[INFO] P:", P, "| defaultParallelism:", spark.sparkContext.defaultParallelism)

# Done shards
done = set(r["sent_shard"] for r in spark.table(T_SENT_PROGRESS).select("sent_shard").distinct().collect())
print(f"[INFO] already done shards: {len(done)} / {N_SENT_SHARDS}")

# Optional health: show biggest shards
print("[QA] Largest sentiment shards (top 10)")
display(
    spark.table(T_SENT_INPUT)
         .groupBy("sent_shard")
         .count()
         .orderBy(F.desc("count"))
         .limit(10)
)

# Main loop
for k in range(N_SENT_SHARDS):
    if k in done:
        continue

    df_k = (
        spark.table(T_SENT_INPUT)
             .filter(F.col("sent_shard") == F.lit(int(k)))
             .select("sentence_id","sentence")
    )

    n_in = df_k.count()
    print(f"\n[RUN] sent_shard={k} | input rows={n_in:,}")
    if n_in == 0:
        spark.createDataFrame([(k, datetime.utcnow().isoformat(), 0)], progress_schema) \
            .write.mode("append").format("delta").saveAsTable(T_SENT_PROGRESS)
        continue

    scored = (
        df_k.coalesce(P)
            .mapInPandas(fn_sst2, schema=sent_schema)
    )

    scored.write.mode("append").format("delta").saveAsTable(T_SENTIMENT_UNIQUE)

    spark.createDataFrame([(k, datetime.utcnow().isoformat(), n_in)], progress_schema) \
         .write.mode("append").format("delta").saveAsTable(T_SENT_PROGRESS)

    # light prints: do NOT count the whole output table each shard
    if (k % 10) == 0:
        done_now = spark.table(T_SENT_PROGRESS).select("sent_shard").distinct().count()
        out_rows = spark.table(T_SENT_PROGRESS).agg(F.sum("rows_scored").alias("rows")).collect()[0]["rows"]
        print(f"[OK] shard {k} appended | approx_rows_scored={out_rows:,} | shards_done={done_now}/{N_SENT_SHARDS}")
    else:
        print(f"[OK] shard {k} appended | shards_done=? (see every 10 shards)")


[OK] Init output: airbnb_sentiment_unique_ITALY_1
[OK] Progress exists (resume): airbnb_sent_progress_ITALY_1
[INFO] P: 8 | defaultParallelism: 4
[INFO] already done shards: 0 / 40
[QA] Largest sentiment shards (top 10)


sent_shard,count
23,21499
10,21403
9,21383
19,21354
36,21337
16,21334
0,21306
28,21303
31,21301
39,21300



[RUN] sent_shard=0 | input rows=21,306
[OK] shard 0 appended | approx_rows_scored=21,306 | shards_done=1/40

[RUN] sent_shard=1 | input rows=21,259
[OK] shard 1 appended | shards_done=? (see every 10 shards)

[RUN] sent_shard=2 | input rows=21,214
[OK] shard 2 appended | shards_done=? (see every 10 shards)

[RUN] sent_shard=3 | input rows=21,158
[OK] shard 3 appended | shards_done=? (see every 10 shards)

[RUN] sent_shard=4 | input rows=20,959
[OK] shard 4 appended | shards_done=? (see every 10 shards)

[RUN] sent_shard=5 | input rows=21,131
[OK] shard 5 appended | shards_done=? (see every 10 shards)

[RUN] sent_shard=6 | input rows=21,160
[OK] shard 6 appended | shards_done=? (see every 10 shards)

[RUN] sent_shard=7 | input rows=21,281
[OK] shard 7 appended | shards_done=? (see every 10 shards)

[RUN] sent_shard=8 | input rows=21,148
[OK] shard 8 appended | shards_done=? (see every 10 shards)

[RUN] sent_shard=9 | input rows=21,383
[OK] shard 9 appended | shards_done=? (see every 10

### Build NEG_UNIQUE (neg_prob >= threshold)

In [0]:
from pyspark.sql import functions as F

sent = spark.table(T_SENTIMENT_UNIQUE)

neg_unique = (
    sent.filter(F.col("neg_prob") >= F.lit(float(NEG_PROB_THRESHOLD)))
        .select("sentence_id","neg_prob","pos_prob","sentiment_label","sent_model")
        .dropDuplicates(["sentence_id"])
)

save_table_and_path(neg_unique, T_NEG_UNIQUE)
print("[INFO] neg_unique rows:", spark.table(T_NEG_UNIQUE).count())
display(neg_unique.select("neg_prob").summary("min","25%","50%","mean","75%","max"))


[OK] Saved table=airbnb_negative_unique_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_negative_unique_ITALY_1
[INFO] neg_unique rows: 171440


summary,neg_prob
min,0.4000253
25%,0.97011423
50%,0.994991
mean,0.9482496823712216
75%,0.9990055
max,0.99982774


## Step 5 - Join: (Issue candidates) × (Negative sentiment) → final issues per sentence
Then map back to occurrences to preserve property/host linkage.

In [0]:
issues = spark.table(T_ISSUES_CANDIDATES_UNIQUE)
neg = spark.table(T_NEG_UNIQUE)

df_final_issues_unique = (
    issues.join(neg.select("sentence_id","neg_prob","pos_prob","sentiment_label"), on="sentence_id", how="inner")
)

save_table_and_path(df_final_issues_unique, T_ISSUES_UNIQUE)
iu2 = spark.table(T_ISSUES_UNIQUE)
print("[OK] Final issues unique saved:", T_ISSUES_UNIQUE, "| rows:", iu2.count())

print("[OK] Final issues unique saved (overwrites candidates):", T_ISSUES_UNIQUE, "| rows:", iu2.count())
display(iu2.groupBy("issue").count().orderBy(F.desc("count")).limit(30))
display(iu2.orderBy(F.desc("neg_prob")).select("neg_prob","issue","issue_sim","sentence").limit(30))

# Map to occurrences
occ = spark.table(T_SENTENCES_CANDIDATES_OCCUR)

df_issues_occur = occ.join(iu2.select("sentence_id","issue","issue_sim","is_top2","top_gap","top1_issue","top2_issue","top1_sim","top2_sim","neg_prob"), on="sentence_id", how="inner")

save_table_and_path(df_issues_occur, T_ISSUES_OCCUR)

io = spark.table(T_ISSUES_OCCUR).cache()
print("[OK] Saved:", T_ISSUES_OCCUR, "| rows:", io.count())
display(io.groupBy("issue").count().orderBy(F.desc("count")).limit(30))
display(io.orderBy(F.desc("issue_sim")).select(PROPERTY_COL, SELLER_ID_COL, "issue", "issue_sim", "neg_prob", "sentence").limit(30))

[OK] Saved table=airbnb_issues_unique_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_issues_unique_ITALY_1
[OK] Final issues unique saved: airbnb_issues_unique_ITALY_1 | rows: 196406
[OK] Final issues unique saved (overwrites candidates): airbnb_issues_unique_ITALY_1 | rows: 196406


issue,count
space_and_privacy,45430
noise,24745
location,16075
parking,14781
cleanliness,11327
accessibility,11135
linens,10797
bathroom_and_water,10497
kitchen_and_appliances,10181
climate_control,10128


neg_prob,issue,issue_sim,sentence
0.99982774,property_condition,0.442354,Cons: Outdated and poorly maintained furniture.
0.99982774,furniture_and_comfort,0.42898405,Cons: Outdated and poorly maintained furniture.
0.9998265,space_and_privacy,0.4153528,"in short, this house is totally poorly managed!!"
0.9998265,property_condition,0.4238606,"in short, this house is totally poorly managed!!"
0.99982566,property_condition,0.48596498,"In addition, the house or furniture is outdated."
0.9998254,noise,0.56116956,The biggest downside of the apartment is the complete lack of soundproofing from the neighbors.
0.9998254,space_and_privacy,0.56955147,The biggest downside of the apartment is the complete lack of soundproofing from the neighbors.
0.99982506,space_and_privacy,0.46569657,Our stay in the apartment was ruined by our very negative experience with its host Barbara and by spending so much time and money on parking which should have been included as it was stated in the add.
0.99982506,parking,0.44855595,Our stay in the apartment was ruined by our very negative experience with its host Barbara and by spending so much time and money on parking which should have been included as it was stated in the add.
0.99982494,cleanliness,0.40477726,a small downside: dishes and one of the bathrooms a little outdated.


[OK] Saved table=airbnb_issues_occurrences_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_issues_occurrences_ITALY_1
[OK] Saved: airbnb_issues_occurrences_ITALY_1 | rows: 195780


issue,count
space_and_privacy,45295
noise,24729
location,16026
parking,14690
cleanliness,11282
accessibility,11108
linens,10758
bathroom_and_water,10458
kitchen_and_appliances,10132
climate_control,10100


property_id,seller_id,issue,issue_sim,neg_prob,sentence
494629,1438903,accessibility,0.8086476,0.99953663,"Not accessible for anyone with mobility problems as there are several stairs to cross from any direction, and there is one flight of stairs to get to the apartment."
648233952734091881,401843367,host_communication,0.8001241,0.9997942,Bad communication with the host and lack of willingness to help.
49378840,217442670,linens,0.7985994,0.9996866,Bed linen and towels of poor quality.
758639842429336426,317057774,accessibility,0.78752816,0.994862,"There are some flights of stairs to get to the apartment, so people with mobility difficulties may have problems."
20141836,36317882,accessibility,0.7866302,0.68929535,"One thing to note: there are stairs up to the apartment, which could be a challenge for those with mobility issues."
28608335,174176425,accessibility,0.786057,0.9995832,There is a difficulty in getting up to the apartment due to stairs and not suitable for older people or with mobility needs.
1126579899638766534,392974362,noise,0.78524446,0.99973744,"The bedroom faces the street, the windows do not have good sound insulation, the loud noises outside on the first night did not stop until four in the morning, the poor sound insulation is not suitable for people who need to be quiet to sleep."
15771001,101986443,noise,0.7843896,0.9969695,Our major issue was the loud noises outside much of the night and minimal noise insulation which kept us up
1131150127667511489,41654619,noise,0.7837426,0.9997999,"Poor noise insulation, especially at night (we could hear the neighbors as if they lived in the apartment)."
16614660,14134639,accessibility,0.7819682,0.9997725,Would not be suitable for anyone with mobility issues due to stairs required to access the apartment.


## Step 6 - Aggregations (Property + Host) + Evidence tables

In [0]:
df = spark.table(T_ISSUES_OCCUR)

# Property-level aggregation
prop_agg = (
    df.groupBy(PROPERTY_COL, SELLER_ID_COL, "issue")
      .agg(
          F.count("*").alias("mentions"),
          F.avg("issue_sim").alias("avg_issue_sim"),
          F.stddev("issue_sim").alias("std_issue_sim"),
          F.expr("percentile_approx(issue_sim, 0.5)").alias("median_issue_sim"),
          F.avg("neg_prob").alias("avg_neg_prob"),
          F.stddev("neg_prob").alias("std_neg_prob"),
          F.expr("percentile_approx(neg_prob, 0.5)").alias("median_neg_prob"),
          F.max("neg_prob").alias("max_neg_prob"),
      )
      .withColumn("avg_issue_sim", F.round("avg_issue_sim", 3))
      .withColumn("std_issue_sim", F.round("std_issue_sim", 3))
      .withColumn("median_issue_sim", F.round("median_issue_sim", 3))
      .withColumn("avg_neg_prob", F.round("avg_neg_prob", 3))
      .withColumn("std_neg_prob", F.round("std_neg_prob", 3))
      .withColumn("median_neg_prob", F.round("median_neg_prob", 3))
      .withColumn("max_neg_prob", F.round("max_neg_prob", 3))
)

save_table_and_path(prop_agg, T_PROP_ISSUE_AGG)
print("[OK] Saved:", T_PROP_ISSUE_AGG, "| rows:", spark.table(T_PROP_ISSUE_AGG).count())
display(spark.table(T_PROP_ISSUE_AGG).limit(50))

# Host-level aggregation
host_agg = (
    df.groupBy(SELLER_ID_COL, "issue")
      .agg(
          F.count("*").alias("mentions"),
          F.countDistinct(PROPERTY_COL).alias("n_properties"),
          F.avg("issue_sim").alias("avg_issue_sim"),
          F.expr("percentile_approx(issue_sim, 0.5)").alias("median_issue_sim"),
          F.avg("neg_prob").alias("avg_neg_prob"),
      )
      .withColumn("avg_issue_sim", F.round("avg_issue_sim", 3))
      .withColumn("median_issue_sim", F.round("median_issue_sim", 3))
      .withColumn("avg_neg_prob", F.round("avg_neg_prob", 3))
      .orderBy(F.desc("mentions"))
)

save_table_and_path(host_agg, T_HOST_ISSUE_AGG)
print("[OK] Saved:", T_HOST_ISSUE_AGG, "| rows:", spark.table(T_HOST_ISSUE_AGG).count())
display(spark.table(T_HOST_ISSUE_AGG).limit(50))

# Evidence tables
K_EVID = 5

w_prop = Window.partitionBy(PROPERTY_COL, SELLER_ID_COL, "issue") \
               .orderBy(F.desc("issue_sim"), F.desc("neg_prob"))

prop_evid = (df.withColumn("ev_rank", F.row_number().over(w_prop))
      .select(PROPERTY_COL, SELLER_ID_COL, "issue", "ev_rank",
          "issue_sim", "neg_prob", "sentence", "sentence_id",
          "is_top2","top_gap","top1_issue","top2_issue"))

save_table_and_path(prop_evid, T_PROP_ISSUE_EVID)
print("[OK] Saved:", T_PROP_ISSUE_EVID, "| rows:", spark.table(T_PROP_ISSUE_EVID).count())
display(spark.table(T_PROP_ISSUE_EVID).limit(50))

w_host = Window.partitionBy(SELLER_ID_COL, "issue") \
               .orderBy(F.desc("issue_sim"), F.desc("neg_prob"))

host_evid = (df.withColumn("ev_rank", F.row_number().over(w_host))
      .select(SELLER_ID_COL, "issue", "ev_rank",
          PROPERTY_COL, "issue_sim", "neg_prob",
          "sentence", "sentence_id",
          "is_top2","top_gap","top1_issue","top2_issue"))

save_table_and_path(host_evid, T_HOST_ISSUE_EVID)
print("[OK] Saved:", T_HOST_ISSUE_EVID, "| rows:", spark.table(T_HOST_ISSUE_EVID).count())
display(spark.table(T_HOST_ISSUE_EVID).limit(50))


[OK] Saved table=airbnb_property_issue_agg_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_property_issue_agg_ITALY_1
[OK] Saved: airbnb_property_issue_agg_ITALY_1 | rows: 151742


property_id,seller_id,issue,mentions,avg_issue_sim,std_issue_sim,median_issue_sim,avg_neg_prob,std_neg_prob,median_neg_prob,max_neg_prob
1000351,5499873,space_and_privacy,2,0.511,0.14,0.412,0.909,0.128,0.818,0.999
1000351,5499873,view,1,0.414,null,0.414,0.999,null,0.999,0.999
1000370211425988972,349117830,rules_and_access,1,0.516,null,0.516,0.998,null,0.998,0.998
1000426725683502915,463531440,parking,1,0.447,null,0.447,0.724,null,0.724,0.724
1000442,5500056,cleanliness,1,0.45,null,0.45,0.957,null,0.957,0.957
1000734026102627315,56317684,noise,3,0.558,0.049,0.549,0.989,0.012,0.992,0.999
1000738082742139223,505325871,furniture_and_comfort,1,0.491,null,0.491,1.0,null,1.0,1.0
1000773211046164351,541566652,cleanliness,1,0.493,null,0.493,0.998,null,0.998,0.998
10009859,20256253,cleanliness,1,0.495,null,0.495,1.0,null,1.0,1.0
10009859,20256253,space_and_privacy,4,0.442,0.048,0.414,0.999,0.0,0.999,1.0


[OK] Saved table=airbnb_host_issue_agg_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_host_issue_agg_ITALY_1
[OK] Saved: airbnb_host_issue_agg_ITALY_1 | rows: 127562


seller_id,issue,mentions,n_properties,avg_issue_sim,median_issue_sim,avg_neg_prob
163663206,space_and_privacy,149,80,0.482,0.47,0.981
96122546,space_and_privacy,122,68,0.481,0.466,0.961
9025189,space_and_privacy,92,46,0.49,0.475,0.967
23532561,space_and_privacy,86,34,0.481,0.471,0.956
23904874,space_and_privacy,85,35,0.496,0.487,0.967
163663206,noise,83,44,0.522,0.508,0.974
27693585,space_and_privacy,78,42,0.486,0.47,0.969
96122546,noise,76,42,0.518,0.513,0.951
486694829,space_and_privacy,75,33,0.492,0.488,0.97
9632167,space_and_privacy,72,52,0.462,0.457,0.955


[OK] Saved table=airbnb_property_issue_evidence_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_property_issue_evidence_ITALY_1
[OK] Saved: airbnb_property_issue_evidence_ITALY_1 | rows: 195780


property_id,seller_id,issue,ev_rank,issue_sim,neg_prob,sentence,sentence_id,is_top2,top_gap,top1_issue,top2_issue
1000351,5499873,space_and_privacy,1,0.609771,0.9990823,"Apartment on the street, like a shop, noisy and with little air.",7f65bb677dd08e3605bf08a598d95e7f4e5481945ef000a21ac95fd3ba4d6ef9,false,0.038367152,space_and_privacy,noise
1000351,5499873,space_and_privacy,2,0.41187993,0.81792736,It was also great to have a basic breakfast left in the apartment.,9c0a241033199eac5f73fc4c00fc805973a41ffcda8918f9ac6a35ecd25539ed,false,0.1601193,space_and_privacy,workspace
1000351,5499873,view,1,0.41380352,0.999213,We sought to book the place in the photos but because it was unavailable stayed in another older property in the centre of town.,8591740b80b5e727b837478a6c94b008a94b1529dff8c09e1f053e0c20fcf43c,true,0.0075575113,rules_and_access,view
1000370211425988972,349117830,rules_and_access,1,0.51591843,0.9976381,"And the second day, three people entered the grounds to view property(I think it was on sale) but we were not informed of this beforehand.",9333078d58fa95a738042b818db561deeaced8127fa83d70e24c65604162c66c,false,0.10730371,rules_and_access,view
1000426725683502915,463531440,parking,1,0.44706622,0.7240208,It was close to the fairground and there was parking.,7d2c94a513305255bd049f0b2b91c7ca97c2c304e4ea9b58f5deb6ab86c178f9,false,0.1836513,parking,location
1000442,5500056,cleanliness,1,0.44984993,0.95705926,Everyday cleaning was included and we found our rooms to be clean at the time of our arrival.,61760de13e0eee807a3079edf026acb81bf2ed44f80dd1fdb61070e025f19fd7,false,0.067035645,cleanliness,linens
1000734026102627315,56317684,noise,1,0.6107538,0.99884677,"Unfortunately, the problem is the loud noise coming from the street (no double glazing or fixtures) all day and night and the neighbors upstairs are extremely noisy.",9470f6a700dbf732b519a8e7b207302d838c3bde9ce167e4833497611936b9d0,false,0.2174654,noise,accessibility
1000734026102627315,56317684,noise,2,0.5490107,0.9748839,"On the other hand, the building is located on a very busy artery which causes noise even with the windows closed.",d2fe960c4c83a4c28799058c5487a12e8252e0e8a5804c79f27ab0f82d2f9aa8,false,0.24287981,noise,space_and_privacy
1000734026102627315,56317684,noise,3,0.5143489,0.99178046,"However, it is located on a busy and therefore somewhat noisy street.",b787fdb33e8ad277f8c38097558cc6a174a1c150b4d7117fb04eb285c159f125,false,0.19547647,noise,location
1000738082742139223,505325871,furniture_and_comfort,1,0.49058494,0.9997154,Sofa in the living room to be changed in fairly poor condition.,8d92601ef9cd5718f68356088770fcdb7226aab2633c83762b7ded1e32d3de9b,false,0.08978957,furniture_and_comfort,space_and_privacy


[OK] Saved table=airbnb_host_issue_evidence_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_host_issue_evidence_ITALY_1
[OK] Saved: airbnb_host_issue_evidence_ITALY_1 | rows: 195780


seller_id,issue,ev_rank,property_id,issue_sim,neg_prob,sentence,sentence_id,is_top2,top_gap,top1_issue,top2_issue
100016484,linens,1,735321625327597438,0.44795975,0.99941707,"In my opinion, it would be necessary to review both the kitchen and table equipment and all the linens for the beds and for the bathrooms, the pillows are really uncomfortable.",266e8e386fca0e0d304fc036fe705d3d9534847c7edbd5b9f95ea5cd7f9ac112,true,0.010872275,furniture_and_comfort,linens
100016484,space_and_privacy,1,735321625327597438,0.5235001,0.9433493,"You can see that it is a \""lived\"" apartment where you feel at home.",48b97aa2c2f0b983fd4d1880e38d37918cee8d40e21dabdd4161fb0ef507c8e5,false,0.20927203,space_and_privacy,cleanliness
100016484,space_and_privacy,2,735321625327597438,0.5126201,0.9532947,"pleasant stay, abundant utensils in the kitchen; however, the lack of availability of spaces for living clothes and more (the few available accommodations are already occupied almost completely);",9dc656c66892a3b6ad5fc266f0cc2aabb11d4452e9a606efaeb6634beeee83ce,false,0.014701605,space_and_privacy,cleanliness
100016484,tv_and_entertainment,1,735321625327597438,0.42340577,0.9926847,"For future guests if you are interested in TV, know that you only see foreign channels.",10845dd3c2a46bd0e9cccc5e4723c4c0584557537133fd1cfc1fc683e99b48ef,false,0.13074964,tv_and_entertainment,view
100018893,cleanliness,1,39714917,0.48216498,0.99514884,Very clean and sterile stay in a building that seems to be all short term stay businesses.,a77b1427425a40f5b3218044c6716a4e2a763ad745637342c18fc3634ae399ec,false,0.104590416,cleanliness,space_and_privacy
100018893,host_communication,1,39714917,0.41548544,0.9988689,"And only when we where facing the problem, the host said she could call the maintenance, but 3 (three) days later!",7eef533c2e135db5cf14e3ce37dd72a0b80c78036d8c9bb6439b8801eb4cfdef,false,0.10241872,host_communication,property_condition
100018893,view,1,39714917,0.6052421,0.8574376,Did not realise that the view is off the other apartments.,d8c55a290c4a061c288bea22407bb1e69c5d45223b36acac3cb527e8c9abf9ff,false,0.15964454,view,space_and_privacy
100020000,cleanliness,1,15544768,0.46588114,0.99976665,In the end we managed to get into the apartment but we were let down with the lack of help from the cleaning lady who was meant to be our point of contact as she was not very responsive or helpful throughout the entire day!,78170b52c71f10d05401e823e2eda1fa9e74a873e052c50983fe6036eeeb3b0e,false,0.011532724,cleanliness,space_and_privacy
10011143,cleanliness,1,837593994238695724,0.5670972,0.9997664,"In the bathroom, the cleanliness was unfortunately not sufficient.",213210b02c550315fade5d11eba1b596d315fd812e2b0f149aa4fad7a272987f,false,0.09720865,cleanliness,bathroom_and_water
10011143,cleanliness,2,5201029,0.5334101,0.9997929,"Negatives: lack of amenities meant to be present such as hand soap, shower gel, shampoo.",6f43514be0a584b2924ec5bfe6b3a63369be39781fbbac6d840917a8755c5c3b,false,0.010442674,cleanliness,bathroom_and_water


## Step 7 - Global summaries (Top issues, most problematic properties/hosts)

In [0]:
df_prop = spark.table(T_PROP_ISSUE_AGG)
df_host = spark.table(T_HOST_ISSUE_AGG)

print("[REPORT] Global top issues (by mentions)")
global_issues = (
    df_prop.groupBy("issue")
           .agg(
               F.sum("mentions").alias("mentions"),
               F.countDistinct(PROPERTY_COL).alias("n_properties"),
               F.avg("avg_neg_prob").alias("avg_neg_prob"),
               F.avg("avg_issue_sim").alias("avg_issue_sim"),
           )
           .withColumn("avg_neg_prob", F.round("avg_neg_prob", 3))
           .withColumn("avg_issue_sim", F.round("avg_issue_sim", 3))
           .orderBy(F.desc("mentions"))
)
display(global_issues.limit(30))

print("[REPORT] Most problematic properties (high mentions + high neg_prob)")
worst_props = (
    df_prop.withColumn("red_flag", F.col("mentions") * F.col("avg_neg_prob") * F.col("avg_issue_sim"))
           .orderBy(F.desc("red_flag"))
)
display(worst_props.limit(50))

print("[REPORT] Most problematic hosts (high mentions across many properties)")
worst_hosts = (
    df_host.withColumn("red_flag", F.col("mentions") * F.col("avg_neg_prob") * F.col("avg_issue_sim"))
           .orderBy(F.desc("red_flag"))
)
display(worst_hosts.limit(50))

example = worst_props.select(PROPERTY_COL, SELLER_ID_COL).limit(1).collect()
if example:
    pid = example[0][PROPERTY_COL]
    sid = example[0][SELLER_ID_COL]
    print(f"[REPORT] Example property drilldown: property_id={pid} | seller_id={sid}")

    display(
        spark.table(T_PROP_ISSUE_AGG)
             .filter((F.col(PROPERTY_COL) == pid) & (F.col(SELLER_ID_COL) == sid))
             .orderBy(F.desc("mentions"), F.desc("avg_neg_prob"))
    )
    display(
        spark.table(T_PROP_ISSUE_EVID)
             .filter((F.col(PROPERTY_COL) == pid) & (F.col(SELLER_ID_COL) == sid))
             .orderBy("issue","ev_rank")
    )

print("[OK] Pipeline complete. All tables saved with RUN_ID:", RUN_ID)

[REPORT] Global top issues (by mentions)


issue,mentions,n_properties,avg_neg_prob,avg_issue_sim
space_and_privacy,45295,29280,0.937,0.482
noise,24729,17173,0.95,0.508
location,16026,13662,0.902,0.457
parking,14690,11390,0.922,0.513
cleanliness,11282,8979,0.97,0.478
accessibility,11108,8554,0.949,0.484
linens,10758,8808,0.963,0.474
bathroom_and_water,10458,8303,0.979,0.47
kitchen_and_appliances,10132,8874,0.936,0.474
climate_control,10100,8381,0.969,0.472


[REPORT] Most problematic properties (high mentions + high neg_prob)


property_id,seller_id,issue,mentions,avg_issue_sim,std_issue_sim,median_issue_sim,avg_neg_prob,std_neg_prob,median_neg_prob,max_neg_prob,red_flag
615098102383546795,270230010,space_and_privacy,11,0.497,0.082,0.471,0.981,0.043,0.996,1.0,5.363127
597628349453826690,114492030,noise,9,0.53,0.054,0.539,0.973,0.064,0.994,1.0,4.64121
41873882,114253061,noise,9,0.53,0.066,0.535,0.959,0.095,0.994,0.999,4.57443
37642837,42753114,parking,9,0.506,0.08,0.47,0.996,0.003,0.997,1.0,4.5357840000000005
16576803,20503714,space_and_privacy,9,0.517,0.053,0.538,0.939,0.141,0.998,1.0,4.369166999999999
648989651454955733,249850421,noise,8,0.533,0.051,0.509,0.993,0.006,0.994,0.999,4.234152
30175366,62074527,accessibility,8,0.536,0.086,0.545,0.976,0.053,0.996,0.999,4.185088
16329537,17023800,space_and_privacy,8,0.526,0.063,0.5,0.987,0.03,0.998,1.0,4.153296
28541452,215410723,linens,8,0.522,0.063,0.499,0.993,0.019,0.999,1.0,4.146768
11157955,8751326,space_and_privacy,8,0.555,0.076,0.521,0.931,0.188,0.999,1.0,4.133640000000001


[REPORT] Most problematic hosts (high mentions across many properties)


seller_id,issue,mentions,n_properties,avg_issue_sim,median_issue_sim,avg_neg_prob,red_flag
163663206,space_and_privacy,149,80,0.482,0.47,0.981,70.453458
96122546,space_and_privacy,122,68,0.481,0.466,0.961,56.393401999999995
9025189,space_and_privacy,92,46,0.49,0.475,0.967,43.59236
163663206,noise,83,44,0.522,0.508,0.974,42.199524000000004
23904874,space_and_privacy,85,35,0.496,0.487,0.967,40.768719999999995
23532561,space_and_privacy,86,34,0.481,0.471,0.956,39.545896
96122546,noise,76,42,0.518,0.513,0.951,37.438968
27693585,space_and_privacy,78,42,0.486,0.47,0.969,36.732851999999994
486694829,space_and_privacy,75,33,0.492,0.488,0.97,35.793
96122546,linens,68,45,0.474,0.464,0.996,32.103072


[REPORT] Example property drilldown: property_id=615098102383546795 | seller_id=270230010


property_id,seller_id,issue,mentions,avg_issue_sim,std_issue_sim,median_issue_sim,avg_neg_prob,std_neg_prob,median_neg_prob,max_neg_prob
615098102383546795,270230010,space_and_privacy,11,0.497,0.082,0.471,0.981,0.043,0.996,1.0
615098102383546795,270230010,kitchen_and_appliances,2,0.466,0.001,0.465,0.999,0.0,0.999,0.999
615098102383546795,270230010,linens,1,0.401,null,0.401,1.0,null,1.0,1.0
615098102383546795,270230010,furniture_and_comfort,1,0.497,null,0.497,1.0,null,1.0,1.0


property_id,seller_id,issue,ev_rank,issue_sim,neg_prob,sentence,sentence_id,is_top2,top_gap,top1_issue,top2_issue
615098102383546795,270230010,furniture_and_comfort,1,0.49706432,0.99977976,"A horrible old, smelly sofa bed in ruins.",357eaa4ddfccad4b5747c0c9f30ab952270a27d407ad2f226098084d435cab14,false,0.049077034,furniture_and_comfort,linens
615098102383546795,270230010,kitchen_and_appliances,1,0.46699744,0.9992073,In this the stove broke and was replaced with 2 cheap electric plates.,74625e2f4d7e13c143eebcdb1c2492eb4d393d491ed3b29434388e46c92c273b,false,0.13835132,kitchen_and_appliances,property_condition
615098102383546795,270230010,kitchen_and_appliances,2,0.46542922,0.9993748,In this the stove was broken and was replaced with 2 cheap electric plates.,00aec0026454aa1bff9b7ad815c7ddaf214a11fcacb123bf180239ea94bc1468,false,0.16276947,kitchen_and_appliances,property_condition
615098102383546795,270230010,linens,1,0.40075308,0.99964964,An old horrible in bad condition stinking pulling out sofa.,ab677b6b8aa2a98b2b4dd4958c067dc9f52b60a06047194978f1e7805df72834,false,0.028211296,linens,furniture_and_comfort
615098102383546795,270230010,space_and_privacy,1,0.64730084,0.99896383,"In addition, it is added that the place (not the apartment because we were unable to access it) is to be desired sinceit is in a less attractive state in reality.",0fe9252d81d960fc24dea8eb039039f5001d5c394ab71085b82c8239c4ef4a10,false,0.15683123,space_and_privacy,accessibility
615098102383546795,270230010,space_and_privacy,2,0.5823832,0.9962238,The entire apartment was one room about 15m2 small.,4a4813ac6c6731452a90caaa89a7be9ca5ac72da7f9aa7c1adf3ae983663c3df,false,0.28126627,space_and_privacy,accessibility
615098102383546795,270230010,space_and_privacy,3,0.5740111,0.9957496,The whole apartment was one room about 15m2 small.,292c2d6deb94942edfa0c3215647658370e07a6257f8755731e371be39aa5eda,false,0.28284287,space_and_privacy,accessibility
615098102383546795,270230010,space_and_privacy,4,0.5516589,0.99140996,The whole apartment was a small room of about 15m2.,830cfaa7bf2867e0e0c94354a210e3e068292340765f08d3e2eb36097bd9d4b0,false,0.26096365,space_and_privacy,accessibility
615098102383546795,270230010,space_and_privacy,5,0.5092308,0.8522232,"It will also describe with bedroom, which means bed in a private room, which was not the case!!",f4cf984475e25585f834e032fa48b6310b2d8b193cd0443748e8c1647444235e,false,0.13769975,space_and_privacy,furniture_and_comfort
615098102383546795,270230010,space_and_privacy,6,0.47090334,0.9759393,"It's also described as a bedroom, which means a bed in a separate room, but it wasn't like that!!",d4c1434fc27111dfb2b0c09c5b89249027c4aade2d49dd2198ec9ef0f6227228,false,0.18455467,space_and_privacy,furniture_and_comfort


[OK] Pipeline complete. All tables saved with RUN_ID: ITALY_1


# LLM recommendations

In [0]:
# =============================
# GOOGLE API KEY (REQUIRED)
# =============================
# Insert your personal Google Generative AI API key below.
# This key is used only to generate live LLM recommendations
# and is NOT stored or shared.

GOOGLE_API_KEY = ""   # <-- paste your API key here

In [0]:
# API config

# GOOGLE_API_KEY = None
GEMINI_MODEL = "models/gemini-2.5-pro"
API_SLEEP_SEC = 0.25
API_TIMEOUT_SEC = 120

# Output tables
T_API_TARGETS = f"airbnb_api_targets_{RUN_ID}"
T_API_RESULTS = f"airbnb_api_recs_{RUN_ID}"

api_schema = "property_id STRING, seller_id STRING, ts STRING, request_json STRING, response_json STRING, status STRING, error STRING"

if GOOGLE_API_KEY is None:
    print("[WARN] GOOGLE_API_KEY is not set. API calls will fail until you load it from secrets/env.")
print("[INFO] API targets table:", T_API_TARGETS)
print("[INFO] API results table:", T_API_RESULTS)


[INFO] API targets table: airbnb_api_targets_ITALY_1
[INFO] API results table: airbnb_api_recs_ITALY_1


In [0]:
from pyspark.sql import functions as F

PROPERTY_IDS = [
    "1042005770541410920",
    "10566180",
    "11251348",
    "1166472471627354698",
    "1259994760695336460",
    "14410929",
    "1773055",
]

# Evidence table (property-level)
EVID_TABLE = T_PROP_ISSUE_EVID

# Filter evidence to the chosen properties
evid = (
    spark.table(EVID_TABLE)
         .filter(F.col(PROPERTY_COL).cast("string").isin(PROPERTY_IDS))
)

# Keep only Top1 evidence (if table contains both top1/top2 info)
# If your evidence table has an "is_top2" flag, this keeps only top1 rows.
if "is_top2" in evid.columns:
    evid = evid.filter((F.col("is_top2").isNull()) | (F.col("is_top2") == F.lit(False)))

# Build targets (one row per property+seller)
picked = (
    evid.groupBy(PROPERTY_COL, SELLER_ID_COL)
        .agg(F.count("*").alias("survived_sentences"))
        .orderBy(F.desc("survived_sentences"))
)

save_table_and_path(picked, T_API_TARGETS)

display(picked)
print("[OK] Targets saved:", T_API_TARGETS)


[OK] Saved table=airbnb_api_targets_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_api_targets_ITALY_1


property_id,seller_id,survived_sentences
1166472471627354698,466600961,5
11251348,58701710,5
14410929,69243554,2
1773055,97770461,2
1042005770541410920,175434314,2
1259994760695336460,583333839,1
10566180,14138971,1


[OK] Targets saved: airbnb_api_targets_ITALY_1


In [0]:
import json
from pyspark.sql import functions as F

def build_payload_for_property(property_id: str, seller_id: str, max_sentences: int = 120):
    """
    Build a compact payload for the LLM (one property):
    - Input: negative + issue-tagged sentences for (property_id, seller_id)
    - Output: a compact dict with metadata + sentences list
    """
    df = (spark.table(T_ISSUES_OCCUR)
             .filter((F.col(PROPERTY_COL) == str(property_id)) & (F.col(SELLER_ID_COL) == str(seller_id)))
             .orderBy(F.desc("neg_prob"), F.desc("issue_sim"))
             .limit(int(max_sentences))
             .select(
                 PROPERTY_COL, SELLER_ID_COL,
                 "sentence_id", "sentence",
                 "issue", "issue_sim",
                 "neg_prob",
                 "is_top2", "top_gap", "top1_issue", "top2_issue"))

    rows = [r.asDict(True) for r in df.collect()]

    return {
        "entity_type": "property",
        "property_id": str(property_id),
        "seller_id": str(seller_id),
        "sentences": rows
    }

In [0]:
import time, requests
from datetime import datetime

def call_gemini_api(payload: dict, timeout_sec: int = 120):
    if not GOOGLE_API_KEY:
        return False, 0, None, "GOOGLE_API_KEY is not set"

    url = f"https://generativelanguage.googleapis.com/v1beta/{GEMINI_MODEL}:generateContent?key={GOOGLE_API_KEY}"

    prompt_text = (
        "You are an expert hospitality operations consultant.\n\n"
        "Analyze the following Airbnb feedback for ONE property.\n"
        "Each item includes: sentence, neg_prob (probability the sentence is negative), and issue category.\n\n"
        "Return a short summary and practical recommendations for the host.\n\n"
        "OUTPUT FORMAT (strict):\n"
        "Summary: <2-3 concise sentences describing the main problems and their impact>\n"
        "Priority: P0 or P1 (P0 = urgent/critical, P1 = important but not critical)\n"
        "Practical recommendations:\n"
        "- <actionable recommendation 1>\n"
        "- <actionable recommendation 2>\n"
        "- <actionable recommendation 3>\n"
        "(2-6 bullets total; be specific and implementable, e.g., noise -> door seals / double glazing / thick curtains)\n\n"
        "Do NOT return JSON. Do NOT add extra sections or titles. Do NOT cite sentence_ids.\n\n"
        f"DATA:\n{json.dumps(payload, ensure_ascii=False, indent=2)}"
    )

    body = {"contents": [{"role": "user", "parts": [{"text": prompt_text}]}]}
    r = requests.post(url, json=body, timeout=timeout_sec)

    if 200 <= r.status_code < 300:
        return True, r.status_code, r.json(), None
    return False, r.status_code, None, (r.text[:2000] if r.text else "HTTP error")

# Ensure results table exists
if not spark.catalog.tableExists(T_API_RESULTS):
    spark.createDataFrame([], api_schema).write.mode("overwrite").format("delta").saveAsTable(T_API_RESULTS)
    print("[OK] Init API results table:", T_API_RESULTS)

targets = spark.table(T_API_TARGETS).collect()
print("[INFO] API targets:", [(t[PROPERTY_COL], t[SELLER_ID_COL], t["survived_sentences"]) for t in targets])

rows_to_save = []

for t in targets:
    pid = t[PROPERTY_COL]
    sid = t[SELLER_ID_COL]

    payload = build_payload_for_property(pid, sid)

    try:
        ok, code, resp_json, err = call_gemini_api(payload, timeout_sec=int(API_TIMEOUT_SEC))
        rows_to_save.append((
            str(pid), str(sid),
            datetime.utcnow().isoformat(),
            json.dumps(payload, ensure_ascii=False),
            json.dumps(resp_json, ensure_ascii=False) if resp_json is not None else None,
            "ok" if ok else f"http_{code}",
            err
        ))
        print(f"[API] property={pid} seller={sid} -> {'OK' if ok else 'ERR'} ({code})")
    except Exception as e:
        rows_to_save.append((
            str(pid), str(sid),
            datetime.utcnow().isoformat(),
            json.dumps(payload, ensure_ascii=False),
            None,
            "exception",
            repr(e)
        ))
        print(f"[API] property={pid} seller={sid} -> EXCEPTION: {e}")

    time.sleep(float(API_SLEEP_SEC))

df_out = spark.createDataFrame(rows_to_save, api_schema)

# append to metastore table
df_out.write.mode("append").format("delta").saveAsTable(T_API_RESULTS)

# also append to OUTPUT_BASE path
df_out.write.mode("append").format("delta").save(f"{OUTPUT_BASE}/{T_API_RESULTS}")

display(spark.table(T_API_RESULTS).orderBy(F.desc("ts")))
print("[OK] Saved:", T_API_RESULTS, "| appended_rows:", df_out.count())

[INFO] API targets: [('1166472471627354698', '466600961', 5), ('11251348', '58701710', 5), ('14410929', '69243554', 2), ('1773055', '97770461', 2), ('1042005770541410920', '175434314', 2), ('1259994760695336460', '583333839', 1), ('10566180', '14138971', 1)]
[API] property=1166472471627354698 seller=466600961 -> OK (200)
[API] property=11251348 seller=58701710 -> OK (200)
[API] property=14410929 seller=69243554 -> OK (200)
[API] property=1773055 seller=97770461 -> OK (200)
[API] property=1042005770541410920 seller=175434314 -> OK (200)
[API] property=1259994760695336460 seller=583333839 -> OK (200)
[API] property=10566180 seller=14138971 -> OK (200)


property_id seller_id ts request_json response_json status error 10566180 14138971 2026-01-24T11:59:51.280698 {"entity_type": "property", "property_id": "10566180", "seller_id": "14138971", "sentences": [{"property_id": "10566180", "seller_id": "14138971", "sentence_id": "ce153e82a40e8b869a3eaeedfcf7fdc158256293b09496f1dfe37d5ed5dd6f35", "sentence": "Issues include: leaking ceilings, unreliable electricity, and prolonged internet outages.", "issue": "property_condition", "issue_sim": 0.6196478605270386, "neg_prob": 0.9992783665657043, "is_top2": false, "top_gap": 0.09395492076873779, "top1_issue": "property_condition", "top2_issue": "safety"}]} {"candidates": [{"content": {"parts": [{"text": "Summary: The property suffers from critical infrastructure failures, including leaking ceilings, unreliable electricity, and poor internet. These issues create an unsafe and uninhabitable environment, severely impacting the guest experience and posing significant safety risks.\nPriority: P0\nPractical recommendations:\n- Immediately hire a licensed roofer or plumber to find and repair the source of the ceiling leaks, and inspect for/remediate any resulting water damage or mold.\n- Engage a certified electrician to conduct a full diagnostic of the electrical system to identify and fix the cause of the unreliability.\n- Temporarily block the property's booking calendar to prevent new arrivals until all critical repairs are completed and verified.\n- Contact the internet service provider to troubleshoot the connection; if the issue persists, switch to a more reliable provider or install a 4G/5G backup system."}], "role": "model"}, "finishReason": "STOP", "index": 0}], "usageMetadata": {"promptTokenCount": 488, "candidatesTokenCount": 170, "totalTokenCount": 1641, "promptTokensDetails": [{"modality": "TEXT", "tokenCount": 488}], "thoughtsTokenCount": 983}, "modelVersion": "gemini-2.5-pro", "responseId": "N7R0aaSGD43J-8YP3Yaf-A4"} ok null 1259994760695336460 583333839 2026-01-24T11:59:39.048867 {"entity_type": "property", "property_id": "1259994760695336460", "seller_id": "583333839", "sentences": [{"property_id": "1259994760695336460", "seller_id": "583333839", "sentence_id": "f8c37d431fa8f2a44f9d226b1ba8ba966d60f9c939b63d96258fe9c1372007f8", "sentence": "Parking spaces in front of the apartment are free every night in October.", "issue": "parking", "issue_sim": 0.4387954771518707, "neg_prob": 0.543191134929657, "is_top2": false, "top_gap": 0.05499204993247986, "top1_issue": "parking", "top2_issue": "space_and_privacy"}]} {"candidates": [{"content": {"parts": [{"text": "Summary: Guest feedback indicates ambiguity around the parking situation, specifically mentioning its seasonal availability. This lack of clarity can create uncertainty and potential unexpected costs for future guests, negatively impacting their arrival experience.\nPriority: P1\nPractical recommendations:\n- Update the listing description to clearly state the year-round parking policy, including any seasonal costs, time restrictions, and payment methods.\n- Include a detailed \"Parking Guide\" in your digital welcome book with a map of nearby free and paid options.\n- Proactively mention the current parking situation in the pre-arrival automated message to manage guest expectations before they arrive."}], "role": "model"}, "finishReason": "STOP", "index": 0}], "usageMetadata": {"promptTokenCount": 518, "candidatesTokenCount": 123, "totalTokenCount": 1246, "promptTokensDetails": [{"modality": "TEXT", "tokenCount": 518}], "thoughtsTokenCount": 605}, "modelVersion": "gemini-2.5-pro", "responseId": "K7R0abdVrI389Q_4sa-pDw"} ok null 1042005770541410920 175434314 2026-01-24T11:59:29.027586 {"entity_type": "property", "property_id": "1042005770541410920", "seller_id": "175434314", "sentences": [{"property_id": "1042005770541410920", "seller_id": "175434314", "sentence_id": "399a0faf07669167551ef3af57d77122999048ca1a59a16c015a870972ecce1f", "sentence": "Towels not present upon arriva

[OK] Saved: airbnb_api_recs_ITALY_1 | appended_rows: 7


In [0]:
from pyspark.sql import functions as F

PROPERTY_IDS = [
    "1042005770541410920",
    "10566180",
    "11251348",
    "1166472471627354698",
    "1259994760695336460",
    "14410929",
    "1773055",
]

api_ok = (
    spark.table(T_API_RESULTS)
         .filter(F.col("status") == "ok")
         .filter(F.col("property_id").cast("string").isin(PROPERTY_IDS))   # <-- במקום RUN_DATE
         .select(
             F.col("property_id").cast("string").alias("property_id"),
             F.col("seller_id").cast("string").alias("seller_id"),
             F.get_json_object(
                 "response_json",
                 "$.candidates[0].content.parts[0].text"
             ).alias("llm_text"),
             "ts"
         )
)

agg = (
    spark.table(T_PROP_ISSUE_AGG)
         .select(
             F.col(PROPERTY_COL).cast("string").alias("property_id"),
             F.col(SELLER_ID_COL).cast("string").alias("seller_id"),
             "issue",
             "mentions",
             "avg_issue_sim",
             "median_issue_sim",
             "avg_neg_prob",
             "median_neg_prob",
             "max_neg_prob"
         )
)

api_prop_issue = (
    api_ok
    .join(agg, on=["property_id", "seller_id"], how="inner")
)

display(api_prop_issue.orderBy(F.desc("mentions")).limit(50))

evid = (
    spark.table(T_PROP_ISSUE_EVID)
         .select(
             F.col(PROPERTY_COL).cast("string").alias("property_id"),
             F.col(SELLER_ID_COL).cast("string").alias("seller_id"),
             "issue",
             "sentence_id",
             "sentence",
             "neg_prob",
             "issue_sim"
         )
)

api_prop_issue_evid = (
    api_prop_issue
    .join(evid, on=["property_id","seller_id","issue"], how="left")
)

display(api_prop_issue_evid.limit(50))

T_API_PROP_ISSUE = "airbnb_api_property_issue_ITALY_1"
T_API_PROP_ISSUE_EVID = "airbnb_api_property_issue_evidence_ITALY_1"

save_table_and_path(api_prop_issue, T_API_PROP_ISSUE)
save_table_and_path(api_prop_issue_evid, T_API_PROP_ISSUE_EVID)

property_id,seller_id,llm_text,ts,issue,mentions,avg_issue_sim,median_issue_sim,avg_neg_prob,median_neg_prob,max_neg_prob
11251348,58701710,"Summary: Guests report that the Wi-Fi connection is weak, which negatively impacts their stay in a connected world. Feedback also implies that storage may be inconveniently located in hallways rather than in the primary living or sleeping areas. Priority: P0 Practical recommendations: - Test the internet speed. If the signal is weak, install a Wi-Fi mesh system or range extender to ensure strong coverage throughout the property. - If the internet speed itself is slow, contact your provider to troubleshoot the line or upgrade to a higher-speed plan (e.g., fiber optic). - Assess the storage inside the bedroom(s). If insufficient, add a freestanding wardrobe, a garment rack, or a chest of drawers for guest convenience. - Update listing photos to clearly show the new in-room storage solutions and mention ""reliable, high-speed Wi-Fi"" in the description to manage expectations and highlight improvements.",2026-01-24T11:58:49.302875,space_and_privacy,2,0.42,0.411,0.759,0.534,0.984
1042005770541410920,175434314,"Summary: The property is failing to provide basic, essential amenities like towels and toilet paper, leading to a poor guest arrival experience. This, combined with concerns about the unit's small size, suggests significant issues with both operational readiness and expectation management in the listing. Priority: P0 Practical recommendations: - Create and use a mandatory pre-check-in checklist for cleaners to verify that all essential supplies, including a set number of towels per guest and at least two rolls of toilet paper, are present. - Standardize the placement of toiletries like soap and shampoo in a visible, designated spot in the bathroom (e.g., a shower caddy or a tray on the vanity). - Review the listing description and photos to ensure they accurately portray the size of the unit and bathroom. Consider adding phrases like ""cozy space"" or explicitly stating the square footage to better manage expectations for groups of three.",2026-01-24T11:59:29.027586,linens,1,0.538,0.538,0.999,0.999,0.999
1042005770541410920,175434314,"Summary: The property is failing to provide basic, essential amenities like towels and toilet paper, leading to a poor guest arrival experience. This, combined with concerns about the unit's small size, suggests significant issues with both operational readiness and expectation management in the listing. Priority: P0 Practical recommendations: - Create and use a mandatory pre-check-in checklist for cleaners to verify that all essential supplies, including a set number of towels per guest and at least two rolls of toilet paper, are present. - Standardize the placement of toiletries like soap and shampoo in a visible, designated spot in the bathroom (e.g., a shower caddy or a tray on the vanity). - Review the listing description and photos to ensure they accurately portray the size of the unit and bathroom. Consider adding phrases like ""cozy space"" or explicitly stating the square footage to better manage expectations for groups of three.",2026-01-24T11:59:29.027586,space_and_privacy,1,0.543,0.543,0.983,0.983,0.983
1166472471627354698,466600961,"Summary: The property has critical cleanliness failures, with guests arriving to dirty floors and stained bedding, creating a terrible first impression. Unreliable Wi-Fi is also a significant issue, failing to provide a basic amenity that modern travelers depend on. Priority: P0 Practical recommendations: - Overhaul your cleaning protocol: Implement a detailed pre-arrival checklist for your cleaners and conduct periodic spot-checks or require photo verification of key areas (floors, made beds) before every guest arrival. - Immediately inspect all linens and replace any items with permanent stains. Purchase backup sets to ensure a fresh supply is always available. - Contact your Internet Service Provider to diagnose 

property_id,seller_id,issue,llm_text,ts,mentions,avg_issue_sim,median_issue_sim,avg_neg_prob,median_neg_prob,max_neg_prob,sentence_id,sentence,neg_prob,issue_sim
11251348,58701710,space_and_privacy,"Summary: Guests report that the Wi-Fi connection is weak, which negatively impacts their stay in a connected world. Feedback also implies that storage may be inconveniently located in hallways rather than in the primary living or sleeping areas. Priority: P0 Practical recommendations: - Test the internet speed. If the signal is weak, install a Wi-Fi mesh system or range extender to ensure strong coverage throughout the property. - If the internet speed itself is slow, contact your provider to troubleshoot the line or upgrade to a higher-speed plan (e.g., fiber optic). - Assess the storage inside the bedroom(s). If insufficient, add a freestanding wardrobe, a garment rack, or a chest of drawers for guest convenience. - Update listing photos to clearly show the new in-room storage solutions and mention ""reliable, high-speed Wi-Fi"" in the description to manage expectations and highlight improvements.",2026-01-24T11:58:49.302875,2,0.42,0.411,0.759,0.534,0.984,879bd10a470e66b387c468349d2133dfb1c9ee84eeb09a37dee5f2ef46a64925,We were already inside the apartment for the second time.,0.5344797,0.41095412
11251348,58701710,space_and_privacy,"Summary: Guests report that the Wi-Fi connection is weak, which negatively impacts their stay in a connected world. Feedback also implies that storage may be inconveniently located in hallways rather than in the primary living or sleeping areas. Priority: P0 Practical recommendations: - Test the internet speed. If the signal is weak, install a Wi-Fi mesh system or range extender to ensure strong coverage throughout the property. - If the internet speed itself is slow, contact your provider to troubleshoot the line or upgrade to a higher-speed plan (e.g., fiber optic). - Assess the storage inside the bedroom(s). If insufficient, add a freestanding wardrobe, a garment rack, or a chest of drawers for guest convenience. - Update listing photos to clearly show the new in-room storage solutions and mention ""reliable, high-speed Wi-Fi"" in the description to manage expectations and highlight improvements.",2026-01-24T11:58:49.302875,2,0.42,0.411,0.759,0.534,0.984,f15ff3db349b45fa0988d9f27b76174c32a04634ea7ca5732faf1bafc3355dd1,"However, there is plenty of space in the hallway in the huge closets.",0.9835122,0.42837542
11251348,58701710,wifi,"Summary: Guests report that the Wi-Fi connection is weak, which negatively impacts their stay in a connected world. Feedback also implies that storage may be inconveniently located in hallways rather than in the primary living or sleeping areas. Priority: P0 Practical recommendations: - Test the internet speed. If the signal is weak, install a Wi-Fi mesh system or range extender to ensure strong coverage throughout the property. - If the internet speed itself is slow, contact your provider to troubleshoot the line or upgrade to a higher-speed plan (e.g., fiber optic). - Assess the storage inside the bedroom(s). If insufficient, add a freestanding wardrobe, a garment rack, or a chest of drawers for guest convenience. - Update listing photos to clearly show the new in-room storage solutions and mention ""reliable, high-speed Wi-Fi"" in the description to manage expectations and highlight improvements.",2026-01-24T11:58:49.302875,1,0.496,0.496,0.949,0.949,0.949,e756a32e883aa042f5dbcc6fa15873d8493d6d6280eae75fb3d9ac6ee76092b0,"Internet connection a little weak, but we're in the mountains after all!",0.949171,0.49598497
1166472471627354698,466600961,kitchen_and_appliances,"Summary: The property has critical cleanliness failures, with guests arriving to dirty floors and stained bedding, creating a terrible first impression. Unreliable Wi-Fi is also a significant issue, failing to provide a basic amenity that modern travelers depend on. Priority: P0 Practical recommendations: 

[OK] Saved table=airbnb_api_property_issue_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_api_property_issue_ITALY_1
[OK] Saved table=airbnb_api_property_issue_evidence_ITALY_1 | path=dbfs:/FileStore/airbnb/airbnb_api_property_issue_evidence_ITALY_1
